In [360]:
import psycopg2
import re
import pandas as pd
import numpy as np
import math
import statistics

#http://140.116.247.169/phppgadmin/
#for eICU
conn = psycopg2.connect(host = "140.116.247.169",database = "eicu"
                        , user = "postgres", password = "ThisIsIIR110Password")
strsql_timeoffst = '1440'
offset = 240
aki_patient = 19615
all_patient = 181244
cur = conn.cursor()


In [321]:
#get AKI patient's ID and diagnosisoffset
cur.execute("SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid")
akiDiagdf = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','diagnosisoffset'])


#net fluid balance
cur = conn.cursor()
cur.execute("SELECT IO.patientunitstayid,IO.intakeoutputoffset,IO.nettotal FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,intakeoutput IO WHERE TB.patientunitstayid=IO.patientunitstayid AND IO.intakeoutputoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY IO.patientunitstayid,IO.intakeoutputoffset")
IO_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','intakeoutputoffset','nettotal'])
IO_res = IO_res.groupby(['patientunitstayid','intakeoutputoffset']).agg({'nettotal':np.max}).reset_index()
IO_res = IO_res.groupby(['patientunitstayid']).agg({'nettotal':np.sum}).reset_index()  
nettotal = IO_res
nettotal.columns = ['patientunitstayid','nettotal']
nettotal.set_index('patientunitstayid',inplace = True)
nettotal = nettotal.fillna(-1)
nettotal = pd.DataFrame(nettotal,dtype = np.float)
for i in nettotal[(nettotal['nettotal'] != -1) & (nettotal['nettotal'] > 3000)].index.values:
    nettotal.loc[i]['nettotal'] = -1
aki_nettotal = nettotal

print("nettotal Missing Rate:",1 - nettotal[nettotal['nettotal']!=-1]['nettotal'].count()/19615)
nettotal


nettotal Missing Rate: 0.32995156767779765


,nettotal
patientunitstayid,
141266,63.22
141304,1253.00
141314,150.00
141448,0.00
141470,0.00
...,...
3353194,-302.00
3353197,2404.60
3353199,17.44


In [322]:
#potassium
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('potassium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
potassium_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
potassium_res = potassium_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
potassium_size = potassium_res.groupby("patientunitstayid").size()
print("potassium max columns:" + str(max(potassium_size)) + "\n" + "potassium avg columns:" + str(float(statistics.mean(potassium_size))) + "\n" + "potassium median columns:" + str(int(statistics.median(potassium_size))))

#potassium turn continuous data from vertical to horizontal
potassium_res['cumCount'] = potassium_res.groupby(['patientunitstayid']).cumcount()
potassium_1 = potassium_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
potassium_1 = potassium_1.iloc[:,:1]

#potassium let format right
potassium_1 = pd.DataFrame(potassium_1)
potassium_1 = potassium_1.fillna(-1)
potassium_1 = pd.DataFrame(potassium_1,dtype = np.float)
column = ['potassium']
potassium_1.columns = column

i = 0
NextOffset = 0
potassium_dflist = []
potassium_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('potassium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    potassium_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'potassium_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    potassium_tdflist.append(potassium_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    potassium_res = potassium_res.groupby(['patientunitstayid']).agg({'potassium_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    potassium_dflist.append(potassium_res)
    i -= 1

potassium = pd.DataFrame()
potassium = potassium_1
for data in potassium_dflist[0:]:
    potassium = pd.merge(potassium,data,on = 'patientunitstayid',how = 'left')
potassium = potassium.set_index('patientunitstayid')
potassium = potassium.fillna(-1)
for j in potassium.columns:
    potassium.loc[(potassium['%s'%j] > 10), '%s'%j] = -1

i = 1
while i < len(potassium.columns):
    if i == 1:
        j = 2
        while j < len(potassium.columns):
            potassium.loc[((potassium['%s'%potassium.columns[i]] == -1) & (potassium['%s'%potassium.columns[j]] != -1)), '%s'%potassium.columns[i]] = potassium['%s'%potassium.columns[j]]
            j += 1
    elif i < len(potassium.columns) - 1:
        potassium.loc[((potassium['%s'%potassium.columns[i]] == -1) & (potassium['%s'%potassium.columns[i + 1]] != -1)), '%s'%potassium.columns[i]] = (potassium['%s'%potassium.columns[i - 1]] + potassium['%s'%potassium.columns[i + 1]])/2
        potassium.loc[(potassium['%s'%potassium.columns[i]] == -1), '%s'%potassium.columns[i]] = potassium['%s'%potassium.columns[i - 1]]
    else:
        potassium.loc[(potassium['%s'%potassium.columns[i]] == -1), '%s'%potassium.columns[i]] = potassium['%s'%potassium.columns[i - 1]]
    i += 1

aki_potassium = potassium
print('')
for i in potassium.columns:
    print("%s Missing Rate:"%i,1 - potassium[potassium['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,potassium[potassium['%s'%i] == -1]['%s'%i].count())
    print('')

potassium


potassium max columns:38
potassium avg columns:1.7973496308846666
potassium median columns:1

potassium Missing Rate: 0.15758348202905936
potassium Null Count: 2

potassium_1 Missing Rate: 0.15758348202905936
potassium_1 Null Count: 2

potassium_2 Missing Rate: 0.15758348202905936
potassium_2 Null Count: 2

potassium_3 Missing Rate: 0.15758348202905936
potassium_3 Null Count: 2

potassium_4 Missing Rate: 0.15758348202905936
potassium_4 Null Count: 2

potassium_5 Missing Rate: 0.15758348202905936
potassium_5 Null Count: 2

potassium_6 Missing Rate: 0.15758348202905936
potassium_6 Null Count: 2



,potassium,potassium_1,potassium_2,potassium_3,potassium_4,potassium_5,potassium_6
patientunitstayid,,,,,,,
141266,5.2,5.2000,5.2000,5.2000,5.2000,5.2000,5.2000
141304,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,5.8000
141314,8.0,8.0000,8.0000,8.0000,8.0000,8.0000,8.0000
141448,2.8,2.8000,2.8000,2.8000,2.8000,2.8000,2.8000
141470,4.3,4.3000,4.3000,4.3000,4.3000,4.3000,4.3000
...,...,...,...,...,...,...,...
3353190,4.2,4.2000,4.2000,4.2000,4.4000,4.6000,4.6000
3353194,3.4,3.4000,3.3500,3.3000,3.5000,3.5500,3.6000
3353197,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,4.8000


In [323]:
#sodium
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('sodium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
sodium_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
sodium_res = sodium_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
sodium_size = sodium_res.groupby("patientunitstayid").size()
print("sodium max columns:" + str(max(sodium_size)) + "\n" + "sodium avg columns:" + str(float(statistics.mean(sodium_size))) + "\n" + "sodium median columns:" + str(int(statistics.median(sodium_size))))

#sodium turn continuous data from vertical to horizontal
sodium_res['cumCount'] = sodium_res.groupby(['patientunitstayid']).cumcount()
sodium_1 = sodium_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
sodium_1 = sodium_1.iloc[:,:1]

#sodium let format right
sodium_1 = pd.DataFrame(sodium_1)
sodium_1 = sodium_1.fillna(-1)
sodium_1 = pd.DataFrame(sodium_1,dtype = np.float)
column = ['sodium']
sodium_1.columns = column

i = 0
NextOffset = 0
sodium_dflist = []
sodium_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('sodium') AND L.labresult IS NOT NULL  AND L.labresult IS NOT NULL  AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    sodium_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'sodium_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    sodium_tdflist.append(sodium_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    sodium_res = sodium_res.groupby(['patientunitstayid']).agg({'sodium_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    sodium_dflist.append(sodium_res)
    i -= 1

sodium = pd.DataFrame()
sodium = sodium_1
for data in sodium_dflist[0:]:
    sodium = pd.merge(sodium,data,on = 'patientunitstayid',how = 'left')
sodium = sodium.set_index('patientunitstayid')
sodium = sodium.fillna(-1)
for j in sodium.columns:
    sodium.loc[((sodium['%s'%j] > 180) | (sodium['%s'%j] < 100)), '%s'%j] = -1

i = 1
while i < len(sodium.columns):
    if i == 1:
        j = 2
        while j < len(sodium.columns):
            sodium.loc[((sodium['%s'%sodium.columns[i]] == -1) & (sodium['%s'%sodium.columns[j]] != -1)), '%s'%sodium.columns[i]] = sodium['%s'%sodium.columns[j]]
            j += 1
    elif i < len(sodium.columns) - 1:
        sodium.loc[((sodium['%s'%sodium.columns[i]] == -1) & (sodium['%s'%sodium.columns[i + 1]] != -1)), '%s'%sodium.columns[i]] = (sodium['%s'%sodium.columns[i - 1]] + sodium['%s'%sodium.columns[i + 1]])/2
        sodium.loc[(sodium['%s'%sodium.columns[i]] == -1), '%s'%sodium.columns[i]] = sodium['%s'%sodium.columns[i - 1]]
    else:
        sodium.loc[(sodium['%s'%sodium.columns[i]] == -1), '%s'%sodium.columns[i]] = sodium['%s'%sodium.columns[i - 1]]
    i += 1

aki_sodium = sodium
print('')
for i in sodium.columns:
    print("%s Missing Rate:"%i,1 - sodium[sodium['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,sodium[sodium['%s'%i] == -1]['%s'%i].count())
    print('')


sodium


sodium max columns:38
sodium avg columns:1.7440638027913722
sodium median columns:1

sodium Missing Rate: 0.15630894723425948
sodium Null Count: 2

sodium_1 Missing Rate: 0.15630894723425948
sodium_1 Null Count: 2

sodium_2 Missing Rate: 0.15630894723425948
sodium_2 Null Count: 2

sodium_3 Missing Rate: 0.15630894723425948
sodium_3 Null Count: 2

sodium_4 Missing Rate: 0.15630894723425948
sodium_4 Null Count: 2

sodium_5 Missing Rate: 0.15630894723425948
sodium_5 Null Count: 2

sodium_6 Missing Rate: 0.15630894723425948
sodium_6 Null Count: 2



,sodium,sodium_1,sodium_2,sodium_3,sodium_4,sodium_5,sodium_6
patientunitstayid,,,,,,,
141266,135.0,135.0000,135.0000,135.0000,135.0000,135.0000,135.0000
141304,141.0,141.0000,141.0000,141.0000,141.0000,141.0000,140.0000
141314,158.0,158.0000,158.0000,158.0000,158.0000,158.0000,158.0000
141448,138.0,138.0000,138.0000,138.0000,138.0000,138.0000,138.0000
141470,131.0,131.0000,131.0000,131.0000,131.0000,131.0000,131.0000
...,...,...,...,...,...,...,...
3353190,139.0,139.0000,139.0000,139.0000,140.0000,141.0000,141.0000
3353194,139.0,139.0000,140.0000,141.0000,139.0000,139.0000,139.0000
3353197,132.0,132.0000,132.0000,132.0000,132.0000,132.0000,132.0000


In [324]:
#Hct
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('Hct') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
Hct_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
Hct_res = Hct_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
Hct_size = Hct_res.groupby("patientunitstayid").size()
print("Hct max columns:" + str(max(Hct_size)) + "\n" + "Hct avg columns:" + str(float(statistics.mean(Hct_size))) + "\n" + "Hct median columns:" + str(int(statistics.median(Hct_size))))

#Hct turn continuous data from vertical to horizontal
Hct_res['cumCount'] = Hct_res.groupby(['patientunitstayid']).cumcount()
Hct_1 = Hct_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
Hct_1 = Hct_1.iloc[:,:1]

#Hct let format right
Hct_1 = pd.DataFrame(Hct_1)
Hct_1 = Hct_1.fillna(-1)
Hct_1 = pd.DataFrame(Hct_1,dtype = np.float)
column = ['Hct']
Hct_1.columns = column

i = 0
NextOffset = 0
Hct_dflist = []
Hct_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('Hct') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    Hct_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'Hct_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    Hct_tdflist.append(Hct_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    Hct_res = Hct_res.groupby(['patientunitstayid']).agg({'Hct_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    Hct_dflist.append(Hct_res)
    i -= 1

Hct = pd.DataFrame()
Hct = Hct_1
for data in Hct_dflist[0:]:
    Hct = pd.merge(Hct,data,on = 'patientunitstayid',how = 'left')
Hct = Hct.set_index('patientunitstayid')
Hct = Hct.fillna(-1)
for j in Hct.columns:
    Hct.loc[((Hct['%s'%j] > 70) | (Hct['%s'%j] < 10)), '%s'%j] = -1

i = 1
while i < len(Hct.columns):
    if i == 1:
        j = 2
        while j < len(Hct.columns):
            Hct.loc[((Hct['%s'%Hct.columns[i]] == -1) & (Hct['%s'%Hct.columns[j]] != -1)), '%s'%Hct.columns[i]] = Hct['%s'%Hct.columns[j]]
            j += 1
    elif i < len(Hct.columns) - 1:
        Hct.loc[((Hct['%s'%Hct.columns[i]] == -1) & (Hct['%s'%Hct.columns[i + 1]] != -1)), '%s'%Hct.columns[i]] = (Hct['%s'%Hct.columns[i - 1]] + Hct['%s'%Hct.columns[i + 1]])/2
        Hct.loc[(Hct['%s'%Hct.columns[i]] == -1), '%s'%Hct.columns[i]] = Hct['%s'%Hct.columns[i - 1]]
    else:
        Hct.loc[(Hct['%s'%Hct.columns[i]] == -1), '%s'%Hct.columns[i]] = Hct['%s'%Hct.columns[i - 1]]
    i += 1

aki_Hct = Hct
print('')
for i in Hct.columns:
    print("%s Missing Rate:"%i,1 - Hct[Hct['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,Hct[Hct['%s'%i] == -1]['%s'%i].count())
    print('')



Hct


Hct max columns:31
Hct avg columns:1.53514526710403
Hct median columns:1

Hct Missing Rate: 0.18460361967881722
Hct Null Count: 11

Hct_1 Missing Rate: 0.1843487127198572
Hct_1 Null Count: 6

Hct_2 Missing Rate: 0.1843487127198572
Hct_2 Null Count: 6

Hct_3 Missing Rate: 0.1843487127198572
Hct_3 Null Count: 6

Hct_4 Missing Rate: 0.1843487127198572
Hct_4 Null Count: 6

Hct_5 Missing Rate: 0.1843487127198572
Hct_5 Null Count: 6

Hct_6 Missing Rate: 0.1843487127198572
Hct_6 Null Count: 6



,Hct,Hct_1,Hct_2,Hct_3,Hct_4,Hct_5,Hct_6
patientunitstayid,,,,,,,
141266,33.4,33.4000,33.4000,33.4000,33.4000,33.4000,33.4000
141304,28.8,28.8000,28.8000,28.8000,28.8000,28.8000,25.5000
141314,34.1,34.1000,34.1000,34.1000,34.1000,34.1000,34.1000
141448,35.8,35.8000,35.8000,35.8000,35.8000,35.8000,35.8000
141470,42.8,42.8000,42.8000,42.8000,42.8000,42.8000,42.8000
...,...,...,...,...,...,...,...
3353190,34.0,34.0000,34.0000,34.0000,33.5000,33.0000,33.0000
3353194,36.0,36.0000,34.5000,33.0000,31.0000,30.5000,30.0000
3353197,23.0,23.0000,23.0000,23.0000,23.0000,23.0000,23.0000


In [325]:
#BUN
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('BUN') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
BUN_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
BUN_res = BUN_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
BUN_size = BUN_res.groupby("patientunitstayid").size()
print("BUN max columns:" + str(max(BUN_size)) + "\n" + "BUN avg columns:" + str(float(statistics.mean(BUN_size))) + "\n" + "BUN median columns:" + str(int(statistics.median(BUN_size))))

#BUN turn continuous data from vertical to horizontal
BUN_res['cumCount'] = BUN_res.groupby(['patientunitstayid']).cumcount()
BUN_1 = BUN_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
BUN_1 = BUN_1.iloc[:,:1]

#BUN let format right
BUN_1 = pd.DataFrame(BUN_1)
BUN_1 = BUN_1.fillna(-1)
BUN_1 = pd.DataFrame(BUN_1,dtype = np.float)
column = ['BUN']
BUN_1.columns = column

i = 0
NextOffset = 0
BUN_dflist = []
BUN_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('BUN') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    BUN_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'BUN_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    BUN_tdflist.append(BUN_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    BUN_res = BUN_res.groupby(['patientunitstayid']).agg({'BUN_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    BUN_dflist.append(BUN_res)
    i -= 1

BUN = pd.DataFrame()
BUN = BUN_1
for data in BUN_dflist[0:]:
    BUN = pd.merge(BUN,data,on = 'patientunitstayid',how = 'left')
BUN = BUN.set_index('patientunitstayid')
BUN = BUN.fillna(-1)
for j in BUN.columns:
    BUN.loc[((BUN['%s'%j] < 1) | (BUN['%s'%j] > 150)), '%s'%j] = -1
i = 1
while i < len(BUN.columns):
    if i == 1:
        j = 2
        while j < len(BUN.columns):
            BUN.loc[((BUN['%s'%BUN.columns[i]] == -1) & (BUN['%s'%BUN.columns[j]] != -1)), '%s'%BUN.columns[i]] = BUN['%s'%BUN.columns[j]]
            j += 1
    elif i < len(BUN.columns) - 1:
        BUN.loc[((BUN['%s'%BUN.columns[i]] == -1) & (BUN['%s'%BUN.columns[i + 1]] != -1)), '%s'%BUN.columns[i]] = (BUN['%s'%BUN.columns[i - 1]] + BUN['%s'%BUN.columns[i + 1]])/2
        BUN.loc[(BUN['%s'%BUN.columns[i]] == -1), '%s'%BUN.columns[i]] = BUN['%s'%BUN.columns[i - 1]]
    else:
        BUN.loc[(BUN['%s'%BUN.columns[i]] == -1), '%s'%BUN.columns[i]] = BUN['%s'%BUN.columns[i - 1]]
    i += 1

aki_BUN = BUN
print('')
for i in BUN.columns:
    print("%s Missing Rate:"%i,1 - BUN[BUN['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,BUN[BUN['%s'%i] == -1]['%s'%i].count())
    print('')


BUN


BUN max columns:10
BUN avg columns:1.5673030155642023
BUN median columns:1

BUN Missing Rate: 0.17552893193984198
BUN Null Count: 276

BUN_1 Missing Rate: 0.1738975274024981
BUN_1 Null Count: 244

BUN_2 Missing Rate: 0.1738975274024981
BUN_2 Null Count: 244

BUN_3 Missing Rate: 0.1738975274024981
BUN_3 Null Count: 244

BUN_4 Missing Rate: 0.1738975274024981
BUN_4 Null Count: 244

BUN_5 Missing Rate: 0.1738975274024981
BUN_5 Null Count: 244

BUN_6 Missing Rate: 0.1738975274024981
BUN_6 Null Count: 244



,BUN,BUN_1,BUN_2,BUN_3,BUN_4,BUN_5,BUN_6
patientunitstayid,,,,,,,
141266,29.0,29.0000,29.0000,29.0000,29.0000,29.0000,29.0000
141304,41.0,41.0000,41.0000,41.0000,41.0000,41.0000,48.0000
141314,13.0,13.0000,13.0000,13.0000,13.0000,13.0000,13.0000
141448,31.0,31.0000,31.0000,31.0000,31.0000,31.0000,31.0000
141470,33.0,33.0000,33.0000,33.0000,33.0000,33.0000,33.0000
...,...,...,...,...,...,...,...
3353190,19.0,19.0000,19.0000,19.0000,20.5000,22.0000,22.0000
3353194,28.0,28.0000,28.0000,28.0000,30.0000,31.5000,33.0000
3353197,57.0,57.0000,57.0000,57.0000,57.0000,57.0000,57.0000


In [326]:
#pH
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('pH') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
pH_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
pH_res = pH_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
pH_size = pH_res.groupby("patientunitstayid").size()
print("pH max columns:" + str(max(pH_size)) + "\n" + "pH avg columns:" + str(float(statistics.mean(pH_size))) + "\n" + "pH median columns:" + str(int(statistics.median(pH_size))))

#pH turn continuous data from vertical to horizontal
pH_res['cumCount'] = pH_res.groupby(['patientunitstayid']).cumcount()
pH_1 = pH_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
pH_1 = pH_1.iloc[:,:1]

#pH let format right
pH_1 = pd.DataFrame(pH_1)
pH_1 = pH_1.fillna(-1)
pH_1 = pd.DataFrame(pH_1,dtype = np.float)
column = ['pH']
pH_1.columns = column

i = 0
NextOffset = 0
pH_dflist = []
pH_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('pH') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    pH_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'pH_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    pH_tdflist.append(pH_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    pH_res = pH_res.groupby(['patientunitstayid']).agg({'pH_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    pH_dflist.append(pH_res)
    i -= 1

pH = pd.DataFrame()
pH = pH_1
for data in pH_dflist[0:]:
    pH = pd.merge(pH,data,on = 'patientunitstayid',how = 'left')
pH = pH.set_index('patientunitstayid')
pH = pH.fillna(-1)
for j in pH.columns:
    pH.loc[((pH['%s'%j] > 7.8) | (pH['%s'%j] < 6.8)), '%s'%j] = -1

i = 1
while i < len(pH.columns):
    if i == 1:
        j = 2
        while j < len(pH.columns):
            pH.loc[((pH['%s'%pH.columns[i]] == -1) & (pH['%s'%pH.columns[j]] != -1)), '%s'%pH.columns[i]] = pH['%s'%pH.columns[j]]
            j += 1
    elif i < len(pH.columns) - 1:
        pH.loc[((pH['%s'%pH.columns[i]] == -1) & (pH['%s'%pH.columns[i + 1]] != -1)), '%s'%pH.columns[i]] = (pH['%s'%pH.columns[i - 1]] + pH['%s'%pH.columns[i + 1]])/2
        pH.loc[(pH['%s'%pH.columns[i]] == -1), '%s'%pH.columns[i]] = pH['%s'%pH.columns[i - 1]]
    else:
        pH.loc[(pH['%s'%pH.columns[i]] == -1), '%s'%pH.columns[i]] = pH['%s'%pH.columns[i - 1]]
    i += 1

aki_pH = pH
print('')
for i in pH.columns:
    print("%s Missing Rate:"%i,1 - pH[pH['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,pH[pH['%s'%i] == -1]['%s'%i].count())
    print('')


pH


pH max columns:36
pH avg columns:1.9405632115038947
pH median columns:1

pH Missing Rate: 0.5766505225592659
pH Null Count: 41

pH_1 Missing Rate: 0.5754269691562579
pH_1 Null Count: 17

pH_2 Missing Rate: 0.5754269691562579
pH_2 Null Count: 17

pH_3 Missing Rate: 0.5754269691562579
pH_3 Null Count: 17

pH_4 Missing Rate: 0.5754269691562579
pH_4 Null Count: 17

pH_5 Missing Rate: 0.5754269691562579
pH_5 Null Count: 17

pH_6 Missing Rate: 0.5754269691562579
pH_6 Null Count: 17



,pH,pH_1,pH_2,pH_3,pH_4,pH_5,pH_6
patientunitstayid,,,,,,,
141266,7.180,7.1800,7.1800,7.1800,7.3600,7.3600,7.3600
141304,7.320,7.3200,7.3200,7.3200,7.3200,7.3200,7.3200
141314,7.140,7.3100,7.3100,7.3100,7.3100,7.3100,7.3100
141515,7.170,7.1700,7.1700,7.1700,7.1700,7.1700,7.1700
141892,7.310,7.3100,7.3100,7.3100,7.3900,7.4700,7.4200
...,...,...,...,...,...,...,...
3353144,7.444,7.4440,7.4440,7.4440,7.4440,7.4440,7.4440
3353190,7.308,7.3080,7.3080,7.3080,7.3080,7.3080,7.3080
3353194,7.404,7.4040,7.4300,7.4560,7.4060,7.4060,7.4060


In [327]:
#OI
#paO2
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('paO2') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
paO2_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','paO2_labresult'])
paO2_res = paO2_res.groupby(['patientunitstayid','labresultoffset']).agg({'paO2_labresult':np.max}).reset_index()
paO2_size = paO2_res.groupby("patientunitstayid").size()
print("paO2 max columns:" + str(max(paO2_size)) + "\n" + "paO2 avg columns:" + str(sum(paO2_size)/len(paO2_size)) + "\n" + "paO2 missing rate:" + str((19615-len(paO2_size))/19615))

#FiO2
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('FiO2') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
FiO2_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','FiO2_labresult'])
FiO2_res = FiO2_res.groupby(['patientunitstayid','labresultoffset']).agg({'FiO2_labresult':np.max}).reset_index()
FiO2_size = FiO2_res.groupby("patientunitstayid").size()
print("FiO2 max columns:" + str(max(FiO2_size)) + "\n" + "FiO2 avg columns:" + str(sum(FiO2_size)/len(FiO2_size)) + "\n" + "FiO2 missing rate:" + str((19615-len(FiO2_size))/19615))

#pick FiO2 and paO2 at the same time,can use this dataframe to calculate ratio
OI = pd.DataFrame()
OI = pd.merge(paO2_res,FiO2_res,on = ['patientunitstayid','labresultoffset'],how = 'inner')
OI_size = OI.groupby("patientunitstayid").size()
print("paO2/FiO2 max columns:" + str(max(OI_size)) + "\n" + "paO2/FiO2 avg columns:" + str(sum(OI_size)/len(OI_size)) + "\n" + "paO2/FiO2 missing rate:" + str((19615-len(OI_size))/19615))

length_OI = int(statistics.median(OI_size))

i = 0
clean_z = OI['FiO2_labresult'].fillna(-1)
clean_z[clean_z == ''] = -1
OI['FiO2_labresult'] = clean_z
clean_z = OI['FiO2_labresult'].fillna(-1)
clean_z[clean_z ==  0] = -1
OI['FiO2_labresult'] = clean_z
OI['OI'] = OI['paO2_labresult'] / OI['FiO2_labresult']
OI.drop(columns = ['paO2_labresult','FiO2_labresult'], inplace = True)

#OI turn continuous data from vertical to horizontal
OI['cumCount'] = OI.groupby(['patientunitstayid']).cumcount()
OI = OI.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'OI')
OI = OI.iloc[:,:length_OI]

#OI let format right
column = []
i = 0
while i < length_OI:
    column.append("OI_"+"%s"%str(i))
    i += 1
OI.columns = column
OI = OI.fillna(-1)
OI = pd.DataFrame(OI,dtype = np.float)

for j in OI.columns:
    OI.loc[((OI['%s'%j] < 10) & (OI['%s'%j] > 0)), '%s'%j] *= 100

aki_OI = OI

print("OI 0 Missing Rate:",1 - OI[OI['OI_0']!=-1]['OI_0'].count()/19615)
OI


paO2 max columns:36
paO2 avg columns:1.9425615412058508
paO2 missing rate:0.5712974764211063
FiO2 max columns:22
FiO2 avg columns:1.8226093591047814
FiO2 missing rate:0.5990823349477441
paO2/FiO2 max columns:22
paO2/FiO2 avg columns:1.8357142857142856
paO2/FiO2 missing rate:0.6574050471577874
OI 0 Missing Rate: 0.6596992097884272


,OI_0
patientunitstayid,
141266,291.000000
141314,73.750000
141515,283.000000
141892,293.333333
142631,530.000000
...,...
3353144,389.000000
3353190,370.833333
3353194,148.000000


In [328]:
#temperature
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartvalue IS NOT NULL AND N.nursingchartcelltypevalname IN('Temperature (C)') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
temperature_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','nursingchartoffset','nursingchartvalue'])
temperature_res = temperature_res.groupby(['patientunitstayid','nursingchartoffset']).agg({'nursingchartvalue':np.max}).reset_index()
temperature_size = temperature_res.groupby("patientunitstayid").size()
print("temperature max columns:" + str(max(temperature_size)) + "\n" + "temperature avg columns:" + str(float(statistics.mean(temperature_size))) + "\n" + "temperature median columns:" + str(int(statistics.median(temperature_size))))

#temperature turn continuous data from vertical to horizontal
temperature_res['cumCount'] = temperature_res.groupby(['patientunitstayid']).cumcount()
temperature_1 = temperature_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartvalue')
temperature_1 = temperature_1.iloc[:,:1]

#temperature let format right
temperature_1 = pd.DataFrame(temperature_1)
temperature_1 = temperature_1.fillna(-1)
temperature_1 = pd.DataFrame(temperature_1,dtype = np.float)
column = ['temperature']
temperature_1.columns = column

i = 0
NextOffset = 0
temperature_dflist = []
temperature_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartvalue IS NOT NULL AND N.nursingchartcelltypevalname IN('Temperature (C)') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset -"+str(PreOffset)+" ORDER BY N.patientunitstayid,N.nursingchartoffset"
    cur.execute(sql)
    temperature_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','nursingchartoffset'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'temperature_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    temperature_tdflist.append(temperature_res.groupby(['patientunitstayid']).agg({'nursingchartoffset'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index())
    temperature_res = temperature_res.groupby(['patientunitstayid']).agg({'temperature_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    temperature_dflist.append(temperature_res)
    i -= 1

temperature = pd.DataFrame()
temperature = temperature_1
for data in temperature_dflist[0:]:
    temperature = pd.merge(temperature,data,on = 'patientunitstayid',how = 'left')
temperature = temperature.set_index('patientunitstayid')
temperature = pd.DataFrame(temperature,dtype = np.float)
temperature = temperature.fillna(-1)
for j in temperature.columns:
    temperature.loc[((temperature['%s'%j] < 33) | (temperature['%s'%j] > 42)), '%s'%j] = -1

i = 1
while i < len(temperature.columns):
    if i == 1:
        j = 2
        while j < len(temperature.columns):
            temperature.loc[((temperature['%s'%temperature.columns[i]] == -1) & (temperature['%s'%temperature.columns[j]] != -1)), '%s'%temperature.columns[i]] = temperature['%s'%temperature.columns[j]]
            j += 1
    elif i < len(temperature.columns) - 1:
        temperature.loc[((temperature['%s'%temperature.columns[i]] == -1) & (temperature['%s'%temperature.columns[i + 1]] != -1)), '%s'%temperature.columns[i]] = (temperature['%s'%temperature.columns[i - 1]] + temperature['%s'%temperature.columns[i + 1]])/2
        temperature.loc[(temperature['%s'%temperature.columns[i]] == -1), '%s'%temperature.columns[i]] = temperature['%s'%temperature.columns[i - 1]]
    else:
        temperature.loc[(temperature['%s'%temperature.columns[i]] == -1), '%s'%temperature.columns[i]] = temperature['%s'%temperature.columns[i - 1]]
    i += 1

aki_temperature = temperature
print('')
for i in temperature.columns:
    print("%s Missing Rate:"%i,1 - temperature[temperature['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,temperature[temperature['%s'%i] == -1]['%s'%i].count())
    print('')

temperature


temperature max columns:126
temperature avg columns:5.084272646798466
temperature median columns:2

temperature Missing Rate: 0.1482538873311241
temperature Null Count: 238

temperature_1 Missing Rate: 0.14315574815192456
temperature_1 Null Count: 138

temperature_2 Missing Rate: 0.14315574815192456
temperature_2 Null Count: 138

temperature_3 Missing Rate: 0.14315574815192456
temperature_3 Null Count: 138

temperature_4 Missing Rate: 0.14315574815192456
temperature_4 Null Count: 138

temperature_5 Missing Rate: 0.14315574815192456
temperature_5 Null Count: 138

temperature_6 Missing Rate: 0.14315574815192456
temperature_6 Null Count: 138



,temperature,temperature_1,temperature_2,temperature_3,temperature_4,temperature_5,temperature_6
patientunitstayid,,,,,,,
141266,36.3,36.3,36.3,36.3,36.2,36.2,36.2
141304,36.8,38.5,38.5,38.5,38.5,38.5,38.0
141314,35.5,35.5,35.5,35.5,35.5,35.5,35.5
141448,36.8,36.8,36.8,36.8,36.8,36.8,37.5
141470,36.7,36.7,36.7,36.7,36.7,36.7,38.2
...,...,...,...,...,...,...,...
3353194,33.3,33.4,33.4,34.2,35.0,36.3,37.0
3353197,38.4,38.4,38.4,38.1,36.8,36.5,36.5
3353199,37.1,37.1,37.1,37.1,37.1,37.1,37.1


In [329]:
#urine
i = 0
NextOffset = 0
urine_dflist = []
urine_tdflist = []
for i in range(1,1 + int(int(strsql_timeoffst)/offset)):
    PreOffset = NextOffset
    NextOffset = offset*i
    sql = "SELECT IO.patientunitstayid,IO.intakeoutputoffset,IO.cellvaluenumeric FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,intakeoutput IO WHERE TB.patientunitstayid=IO.patientunitstayid AND IO.celllabel IN('Urine') AND IO.intakeoutputoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY IO.patientunitstayid,IO.intakeoutputoffset"
    cur.execute(sql)
    urine_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','intakeoutputoffset_'+str(i),'urine_'+str(i)])
    urine_tdflist.append(urine_res.groupby(['patientunitstayid']).agg({'intakeoutputoffset_'+str(i):np.sum}).reset_index())
    urine_res = urine_res.groupby(['patientunitstayid']).agg({'urine_'+str(i):np.sum}).reset_index() 
    urine_dflist.append(urine_res)


urine = pd.DataFrame()
urine = urine_dflist[0]
for data in urine_dflist[1:]:
    urine = pd.merge(urine,data,on = 'patientunitstayid',how = 'left')
urine = urine.set_index('patientunitstayid')
urine = urine.fillna(-1)
# print(len(urine.index))
# i = 0
# total = []
# while i < len(urine.index):
#     a = 0
#     if urine.iloc[i]['urine_1'] != -1:
#         a += float(urine.iloc[i]['urine_1'])
#     if urine.iloc[i]['urine_2'] != -1:
#         a += float(urine.iloc[i]['urine_2'])
#     if urine.iloc[i]['urine_3'] != -1:
#         a += float(urine.iloc[i]['urine_3'])
#     if urine.iloc[i]['urine_4'] != -1:
#         a += float(urine.iloc[i]['urine_4'])
#     total.append(a)
#     i += 1
# urine['urine_total'] = total
urine = pd.DataFrame(urine,dtype = np.float)

aki_urine = urine

for i in urine.columns:
    print("%s Missing Rate:"%i,1 - urine[urine['%s'%i] != -1]['%s'%i].count()/aki_patient)

urine


urine_1 Missing Rate: 0.6617384654601071
urine_2 Missing Rate: 0.8513892429263319
urine_3 Missing Rate: 0.8850369615090492
urine_4 Missing Rate: 0.9124139689013511
urine_5 Missing Rate: 0.9214376752485343
urine_6 Missing Rate: 0.9370379811368851


,urine_1,urine_2,urine_3,urine_4,urine_5,urine_6
patientunitstayid,,,,,,
141594,300.0,-1.0,-1.0,-1.0,-1.0,-1.0
141786,200.0,-1.0,-1.0,-1.0,-1.0,-1.0
141866,600.0,-1.0,-1.0,-1.0,-1.0,-1.0
141893,310.0,-1.0,-1.0,-1.0,-1.0,-1.0
142129,1250.0,750.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...
3353127,300.0,-1.0,-1.0,-1.0,-1.0,-1.0
3353194,20.0,20.0,30.0,50.0,50.0,305.0
3353197,135.0,175.0,135.0,140.0,130.0,175.0


In [330]:
#MAP
cur.execute("SELECT VP.patientunitstayid,VP.observationoffset,VP.noninvasivediastolic,VP.noninvasivesystolic FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,vitalaperiodic VP WHERE TB.patientunitstayid=VP.patientunitstayid AND noninvasivesystolic IS NOT NULL AND noninvasivesystolic IS NOT NULL AND VP.observationoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,observationoffset")
MAP_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','observationoffset','diastolic','systolic'])
MAP_res = MAP_res.groupby(['patientunitstayid','observationoffset']).agg({'diastolic':np.max,'systolic':np.max}).reset_index()
MAP_size = MAP_res.groupby("patientunitstayid").size()
print("MAP max columns:" + str(max(MAP_size)) + "\n" + "MAP avg columns:" + str(float(statistics.mean(MAP_size))) + "\n" + "MAP median columns:" + str(int(statistics.median(MAP_size))))

#MAP turn continuous data from vertical to horizontal
MAP_res['cumCount'] = MAP_res.groupby(['patientunitstayid']).cumcount()
MAP_1 = MAP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'systolic')
MAP_1 = MAP_1.iloc[:,:1]

#MAP let format right
MAP_1 = pd.DataFrame(MAP_1)
MAP_1 = MAP_1.fillna(-1)
MAP_1 = pd.DataFrame(MAP_1,dtype = np.float)
column = ['MAP']
MAP_1.columns = column

i = 0
NextOffset = 0
MAP_dflist = []
MAP_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql =           "SELECT VP.patientunitstayid,VP.observationoffset,VP.noninvasivediastolic,VP.noninvasivesystolic FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,vitalaperiodic VP WHERE TB.patientunitstayid=VP.patientunitstayid AND noninvasivesystolic IS NOT NULL AND noninvasivesystolic IS NOT NULL AND VP.observationoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset -"+str(PreOffset)+" ORDER BY patientunitstayid,observationoffset"
    cur.execute(sql)
    MAP_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','nursingchartoffset'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'diastolic_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'systolic_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    MAP_tdflist.append(MAP_res.groupby(['patientunitstayid']).agg({'nursingchartoffset'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index())
    MAP_res = MAP_res.groupby(['patientunitstayid']).agg({'diastolic_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max,'systolic_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    MAP_dflist.append(MAP_res)
    i -= 1

MAP = pd.DataFrame()
MAP = MAP_1
for data in MAP_dflist[0:]:
    MAP = pd.merge(MAP,data,on = 'patientunitstayid',how = 'left')
MAP = MAP.set_index('patientunitstayid')
MAP = pd.DataFrame(MAP,dtype = np.float)
MAP = MAP.fillna(-1)

j = 1
while j < int(strsql_timeoffst)/offset + 1:
    MAP.loc[((MAP['diastolic_%s'%j] < 30) | (MAP['diastolic_%s'%j] > 140)), 'diastolic_%s'%j] = -1
    MAP.loc[((MAP['systolic_%s'%j] < 60) | (MAP['systolic_%s'%j] > 200)), 'systolic_%s'%j] = -1
    j += 1

i = 1
while i < int(strsql_timeoffst)/offset + 1:
    if i == 1:
        j = 2
        while j < int(strsql_timeoffst)/offset + 1:
            MAP.loc[((MAP['diastolic_%s'%i] == -1) & (MAP['diastolic_%s'%j] != -1)), 'diastolic_%s'%i] = MAP['diastolic_%s'%j]
            MAP.loc[((MAP['systolic_%s'%i] == -1) & (MAP['systolic_%s'%j] != -1)), 'systolic_%s'%i] = MAP['systolic_%s'%j]
            j += 1
    elif i < int(strsql_timeoffst)/offset:
        MAP.loc[((MAP['diastolic_%s'%i] == -1) & (MAP['diastolic_%s'%(i + 1)] != -1)), 'diastolic_%s'%i] = (MAP['diastolic_%s'%(i - 1)] + MAP['diastolic_%s'%(i + 1)])/2
        MAP.loc[(MAP['diastolic_%s'%i] == -1), 'diastolic_%s'%(i)] = MAP['diastolic_%s'%(i - 1)]
        MAP.loc[((MAP['systolic_%s'%(i)] == -1) & (MAP['systolic_%s'%(i + 1)] != -1)), 'systolic_%s'%(i)] = (MAP['systolic_%s'%(i - 1)] + MAP['systolic_%s'%(i + 1)])/2
        MAP.loc[(MAP['systolic_%s'%i] == -1), 'systolic_%s'%i] = MAP['systolic_%s'%(i - 1)]
    else:
        MAP.loc[(MAP['diastolic_%s'%(i)] == -1), 'diastolic_%s'%(i)] = MAP['diastolic_%s'%(i - 1)]
        MAP.loc[(MAP['systolic_%s'%(i)] == -1), 'systolic_%s'%(i)] = MAP['systolic_%s'%(i - 1)]
    MAP['MAP_%s'%i] = (MAP['systolic_%s'%i] - MAP['diastolic_%s'%i])/3 + MAP['diastolic_%s'%i]
    i += 1

i = 1
while i < int(strsql_timeoffst)/offset + 1:
    MAP.drop(columns = ['diastolic_%s'%i], inplace = True)
    MAP.drop(columns = ['systolic_%s'%i], inplace = True)
    i += 1

MAP.drop(columns = ['MAP'], inplace = True)
aki_MAP = MAP


print('')
for i in MAP.columns:
    print("%s Missing Rate:"%i,1 - MAP[MAP['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,MAP[MAP['%s'%i] == -1]['%s'%i].count())
    print('')

MAP

MAP max columns:1385
MAP avg columns:19.3355091383812
MAP median columns:8

MAP_1 Missing Rate: 0.14529696660718838
MAP_1 Null Count: 87

MAP_2 Missing Rate: 0.14529696660718838
MAP_2 Null Count: 87

MAP_3 Missing Rate: 0.14529696660718838
MAP_3 Null Count: 87

MAP_4 Missing Rate: 0.14529696660718838
MAP_4 Null Count: 87

MAP_5 Missing Rate: 0.14529696660718838
MAP_5 Null Count: 87

MAP_6 Missing Rate: 0.14529696660718838
MAP_6 Null Count: 87



,MAP_1,MAP_2,MAP_3,MAP_4,MAP_5,MAP_6
patientunitstayid,,,,,,
141266,75.666667,75.666667,75.666667,75.666667,75.666667,75.666667
141304,72.000000,72.000000,72.000000,72.000000,72.000000,101.000000
141314,96.666667,96.666667,96.666667,96.666667,96.666667,96.666667
141448,81.666667,81.666667,81.666667,81.666667,81.666667,81.666667
141470,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000
...,...,...,...,...,...,...
3353144,86.333333,86.333333,86.333333,86.333333,86.333333,86.333333
3353190,72.333333,72.333333,72.333333,72.333333,72.333333,101.000000
3353194,85.000000,89.000000,94.333333,82.000000,88.000000,78.666667


In [331]:
#MSI modified shock index
store = []
epoch = int(int(strsql_timeoffst)/offset)
while epoch > 0:
    PreOffset = offset*(epoch - 1)
    NextOffset = offset*epoch
    cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartvalue IS NOT NULL AND N.nursingchartcelltypevalname IN('Heart Rate') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY N.patientunitstayid,N.nursingchartoffset")
    HR_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','nursingchartoffset','HR_nursingchartvalue'])
    cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartvalue IS NOT NULL AND N.nursingchartcelltypevalname IN('Non-Invasive BP Systolic') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY N.patientunitstayid,N.nursingchartoffset")
    SBP_res = pd.DataFrame(cur.fetchall(),columns =[ 'patientunitstayid','nursingchartoffset','SBP_nursingchartvalue'])

    # time offset
    HR_res['cumCount'] = HR_res.groupby(['patientunitstayid']).cumcount()
    HR_time = HR_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')

    SBP_res['cumCount'] = SBP_res.groupby(['patientunitstayid']).cumcount()
    SBP_time = SBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')


    #real value
    HR_res['cumCount'] = HR_res.groupby(['patientunitstayid']).cumcount()
    HR = HR_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'HR_nursingchartvalue')

    SBP_res['cumCount'] = SBP_res.groupby(['patientunitstayid']).cumcount()
    SBP = SBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'SBP_nursingchartvalue')


    HR = HR.fillna(-1)
    SBP = SBP.fillna(-1)
    count = 0

    i = 0
    column = []
    while i < len(HR.columns):
        column.append("HR_%s"%i)
        i += 1
    HR.columns = column

    i = 0
    column = []
    while i < len(SBP.columns):
        column.append("SBP_%s"%i)
        i += 1
    SBP.columns = column

    HR_time = HR_time.fillna(-1)
    SBP_time = SBP_time.fillna(-1)
    count = 0

    i = 0
    column = []
    while i < len(HR_time.columns):
        column.append("HR_offset_%s"%i)
        i += 1
    HR_time.columns = column

    i = 0
    column = []
    while i < len(SBP_time.columns):
        column.append("SBP_offset_%s"%i)
        i += 1
    SBP_time.columns = column


    SBP_size = SBP_res.groupby("patientunitstayid").size()
    HR_size = HR_res.groupby("patientunitstayid").size()

    # length_MSI = #int(int(strsql_timeoffst)/offset)

    MSI = []
    i = 0
    while i < len(SBP.index):
        #one time deal with one patient
        goal = 0
        loop = 0
#         whether index is in DBP SBP HR
        if SBP.index[i] in HR.index.values:
            SIpt = []
            SIpt.append(SBP.index[i])
    #         take DBP columns as basic columns length check loop value under length and success under 9
            while loop < SBP_size.loc[SBP.index[i]] and goal < 1:
                #whether we have every ingredient
                if SBP_time.loc[SBP.index[i]][loop]in HR_time.loc[SBP.index[i]].values:
                    SBP_col_index = loop
                    HR_col_index = int(np.where(HR_time.loc[SBP.index[i]].values == SBP_time.loc[SBP.index[i]][loop])[0][0])
                    #caculate MSI
                    msi = 0
                    pulse = 0
                    MAPp = 0
                    if (float(HR.loc[SBP.index[i]][HR_col_index]) != 0 or -1) and (float(SBP.loc[SBP.index[i]][SBP_col_index]) != 0 or -1): 
                        MAPp = float(SBP.loc[SBP.index[i]][SBP_col_index])
                        if MAPp != 0:
                            msi = float(HR.loc[SBP.index[i]][HR_col_index]) / MAPp
                            SIpt.append(msi)
                            goal += 1
                loop += 1
            MSI.append(SIpt)
        i += 1

    MSI = pd.DataFrame(MSI)
    column = ['patientunitstayid']
    column.append("MSI_%s"%(7 - epoch))
    MSI.columns = column
    MSI.set_index('patientunitstayid',inplace = True)
    MSI = MSI.fillna(-1)
    MSI = pd.DataFrame(MSI,dtype = np.float)
    store.append(MSI)
    epoch -= 1
    print(epoch)

#merge every time period
MSI = store[0].index.values
i = 1
while i < len(store):
    MSI = np.union1d(MSI,store[i].index.values)
    i += 1

MSI = pd.DataFrame(MSI)
MSI.columns = ['patientunitstayid']
MSI.set_index('patientunitstayid',inplace = True)
for i in store:
    MSI = pd.merge(MSI,i,on = 'patientunitstayid',how = 'left')
MSI = MSI.fillna(-1)

i = 0
while i < len(MSI.columns):
    if i == 0:
        j = 1
        while j < len(MSI.columns):
            MSI.loc[((MSI['%s'%MSI.columns[i]] == -1) & (MSI['%s'%MSI.columns[j]] != -1)), '%s'%MSI.columns[i]] = MSI['%s'%MSI.columns[j]]
            j += 1
    elif i < len(MSI.columns) - 1:
        MSI.loc[((MSI['%s'%MSI.columns[i]] == -1) & (MSI['%s'%MSI.columns[i + 1]] != -1)), '%s'%MSI.columns[i]] = (MSI['%s'%MSI.columns[i - 1]] + MSI['%s'%MSI.columns[i + 1]])/2
        MSI.loc[(MSI['%s'%MSI.columns[i]] == -1), '%s'%MSI.columns[i]] = MSI['%s'%MSI.columns[i - 1]]
    else:
        MSI.loc[(MSI['%s'%MSI.columns[i]] == -1), '%s'%MSI.columns[i]] = MSI['%s'%MSI.columns[i - 1]]
    i += 1

aki_MSI = MSI
print('')
for i in MSI.columns:
    print("%s Missing Rate:"%i,1 - MSI[MSI['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,MSI[MSI['%s'%i] == -1]['%s'%i].count())
    print('')

MSI


5
4
3
2
1
0

MSI_1 Missing Rate: 0.17960744328320166
MSI_1 Null Count: 45

MSI_2 Missing Rate: 0.17960744328320166
MSI_2 Null Count: 45

MSI_3 Missing Rate: 0.17960744328320166
MSI_3 Null Count: 45

MSI_4 Missing Rate: 0.17960744328320166
MSI_4 Null Count: 45

MSI_5 Missing Rate: 0.17960744328320166
MSI_5 Null Count: 45

MSI_6 Missing Rate: 0.17960744328320166
MSI_6 Null Count: 45



,MSI_1,MSI_2,MSI_3,MSI_4,MSI_5,MSI_6
patientunitstayid,,,,,,
141266,0.916667,0.916667,0.916667,0.810345,0.910891,0.960000
141304,0.854015,0.854015,0.854015,0.854015,0.854015,0.991597
141314,1.545455,1.545455,1.545455,1.545455,1.545455,1.545455
141448,0.860927,0.860927,0.860927,0.860927,0.860927,0.851351
141470,0.947761,0.947761,0.947761,0.947761,0.947761,1.120000
...,...,...,...,...,...,...
3353190,0.709677,0.709677,0.739837,0.717241,1.218750,0.836879
3353194,0.881720,0.734513,0.685950,1.125000,0.990385,1.147727
3353197,0.796296,0.669291,0.661017,0.706897,0.627273,0.673077


In [332]:
#age,since its missing rate is 0,so I don't print its relative information
cur.execute("SELECT patientunitstayid,age FROM patient WHERE patientunitstayid IN (SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
age_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','age'])
age = age_res
age = np.array(age)
age = age.tolist()
for i in age:
    if i[1].isdigit() == False:
        i[1] = ''.join([x for x in i[1] if x.isdigit()])
        if i[1] != '':
            i[1] = int(i[1])
        else:
            i[1] = -1
    else:
        i[1] = int(i[1])
age = pd.DataFrame(age)
age.columns = ['patientunitstayid','age']
age.set_index('patientunitstayid',inplace = True)
age = pd.DataFrame(age,dtype = np.float)
aki_age =age

age


                    age
patientunitstayid      
141266             73.0
141304             70.0
141314             45.0
141448             65.0
141470             58.0
...                 ...
3353194            51.0
3353197            66.0
3353199            66.0
3353251            73.0
3353254            81.0

[19615 rows x 1 columns]


In [333]:
#weight
cur.execute("SELECT patientunitstayid,admissionweight FROM patient WHERE patientunitstayid IN (SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) AND admissionweight IS NOT NULL ORDER BY patientunitstayid")
weight_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','weight'])
weight_size = weight_res.groupby("patientunitstayid").size()
print("weight max columns:" + str(max(weight_size)) + "\n" + "weight avg columns:" + str(sum(weight_size)/len(weight_size)) + "\n" + "weight missing rate:" + str((19615-len(weight_size))/19615))

#weight turn continuous data from vertical to horizontal
weight_res['cumCount'] = weight_res.groupby(['patientunitstayid']).cumcount()
weight = weight_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'weight')


#weight let format right
weight = weight.fillna(-1)
weight = pd.DataFrame(weight,dtype = np.float)
weight.columns = ['weight']
weight.loc[((weight['weight'] < 1) | (weight['weight'] > 635)), 'weight'] = -1
aki_weight = weight

weight


weight max columns:1
weight avg columns:1.0
weight missing rate:0.04027529951567678
                   weight
patientunitstayid        
141266             120.40
141448              79.80
141515              70.90
141594             108.30
141786             112.00
...                   ...
3353194             63.05
3353197             71.50
3353199             71.50
3353251            102.00
3353254             83.90

[18825 rows x 1 columns]


In [334]:
#albumin
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('albumin') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
albumin_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
albumin_res = albumin_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
albumin_size = albumin_res.groupby("patientunitstayid").size()
print("albumin max columns:" + str(max(albumin_size)) + "\n" + "albumin avg columns:" + str(float(statistics.mean(albumin_size))) + "\n" + "albumin median columns:" + str(int(statistics.median(albumin_size))))

#albumin turn continuous data from vertical to horizontal
albumin_res['cumCount'] = albumin_res.groupby(['patientunitstayid']).cumcount()
albumin_1 = albumin_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
albumin_1 = albumin_1.iloc[:,:1]

#albumin let format right
albumin_1 = pd.DataFrame(albumin_1)
albumin_1 = albumin_1.fillna(-1)
albumin_1 = pd.DataFrame(albumin_1,dtype = np.float)
column = ['albumin']
albumin_1.columns = column

i = 0
NextOffset = 0
albumin_dflist = []
albumin_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('albumin') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    albumin_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'albumin_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    albumin_tdflist.append(albumin_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    albumin_res = albumin_res.groupby(['patientunitstayid']).agg({'albumin_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index() 
    albumin_dflist.append(albumin_res)
    i -= 1

albumin = pd.DataFrame()
albumin = albumin_1
for data in albumin_dflist[0:]:
    albumin = pd.merge(albumin,data,on = 'patientunitstayid',how = 'left')
albumin = albumin.set_index('patientunitstayid')
albumin = albumin.fillna(-1)
for j in albumin.columns:
    albumin.loc[((albumin['%s'%j] < 1) | (albumin['%s'%j] > 10)), '%s'%j] = -1

i = 1
while i < len(albumin.columns):
    if i == 1:
        j = 2
        while j < len(albumin.columns):
            albumin.loc[((albumin['%s'%albumin.columns[i]] == -1) & (albumin['%s'%albumin.columns[j]] != -1)), '%s'%albumin.columns[i]] = albumin['%s'%albumin.columns[j]]
            j += 1
    elif i < len(albumin.columns) - 1:
        albumin.loc[((albumin['%s'%albumin.columns[i]] == -1) & (albumin['%s'%albumin.columns[i + 1]] != -1)), '%s'%albumin.columns[i]] = (albumin['%s'%albumin.columns[i - 1]] + albumin['%s'%albumin.columns[i + 1]])/2
        albumin.loc[(albumin['%s'%albumin.columns[i]] == -1), '%s'%albumin.columns[i]] = albumin['%s'%albumin.columns[i - 1]]
    else:
        albumin.loc[(albumin['%s'%albumin.columns[i]] == -1), '%s'%albumin.columns[i]] = albumin['%s'%albumin.columns[i - 1]]
    i += 1

aki_albumin = albumin
print('')
for i in albumin.columns:
    print("%s Missing Rate:"%i,1 - albumin[albumin['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,albumin[albumin['%s'%i] == -1]['%s'%i].count())
    print('')

albumin



albumin max columns:8
albumin avg columns:1.2022481625594466
albumin median columns:1

albumin Missing Rate: 0.4109609992352792
albumin Null Count: 11

albumin_1 Missing Rate: 0.4108590364516951
albumin_1 Null Count: 9

albumin_2 Missing Rate: 0.4108590364516951
albumin_2 Null Count: 9

albumin_3 Missing Rate: 0.4108590364516951
albumin_3 Null Count: 9

albumin_4 Missing Rate: 0.4108590364516951
albumin_4 Null Count: 9

albumin_5 Missing Rate: 0.4108590364516951
albumin_5 Null Count: 9

albumin_6 Missing Rate: 0.4108590364516951
albumin_6 Null Count: 9



,albumin,albumin_1,albumin_2,albumin_3,albumin_4,albumin_5,albumin_6
patientunitstayid,,,,,,,
141266,3.6,3.6000,3.6000,3.6000,3.6000,3.6000,3.6000
141304,3.0,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000
141314,1.5,1.5000,1.5000,1.5000,1.5000,1.5000,1.5000
141448,3.7,3.7000,3.7000,3.7000,3.7000,3.7000,3.7000
141470,2.8,2.8000,2.8000,2.8000,2.8000,2.8000,2.8000
...,...,...,...,...,...,...,...
3353144,2.6,2.6000,2.6000,2.6000,2.6000,2.6000,2.6000
3353190,4.1,4.1000,4.1000,4.1000,4.1000,4.1000,4.1000
3353194,3.1,3.1000,3.1000,3.1000,3.1000,3.1000,3.1000


In [335]:
#creatinine
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('creatinine') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
creatinine_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
creatinine_res = creatinine_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
creatinine_size = creatinine_res.groupby("patientunitstayid").size()
print("creatinine max columns:" + str(max(creatinine_size)) + "\n" + "creatinine avg columns:" + str(float(statistics.mean(creatinine_size))) + "\n" + "creatinine median columns:" + str(int(statistics.median(creatinine_size))))

#creatinine turn continuous data from vertical to horizontal
creatinine_res['cumCount'] = creatinine_res.groupby(['patientunitstayid']).cumcount()
creatinine_1 = creatinine_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
creatinine_1 = creatinine_1.iloc[:,:1]

#creatinine let format right
creatinine_1 = pd.DataFrame(creatinine_1)
creatinine_1 = creatinine_1.fillna(-1)
creatinine_1 = pd.DataFrame(creatinine_1,dtype = np.float)
column = ['creatinine']
creatinine_1.columns = column

i = 0
NextOffset = 0
creatinine_dflist = []
creatinine_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('creatinine') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    creatinine_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'creatinine_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    creatinine_tdflist.append(creatinine_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    creatinine_res = creatinine_res.groupby(['patientunitstayid']).agg({'creatinine_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index() 
    creatinine_dflist.append(creatinine_res)
    i -= 1

creatinine = pd.DataFrame()
creatinine = creatinine_1
for data in creatinine_dflist[0:]:
    creatinine = pd.merge(creatinine,data,on = 'patientunitstayid',how = 'left')
creatinine = creatinine.set_index('patientunitstayid')
creatinine = creatinine.fillna(-1)
for j in creatinine.columns:
    creatinine.loc[((creatinine['%s'%j] < 0.2) | (creatinine['%s'%j] > 10)), '%s'%j] = -1

i = 1
while i < len(creatinine.columns):
    if i == 1:
        j = 2
        while j < len(creatinine.columns):
            creatinine.loc[((creatinine['%s'%creatinine.columns[i]] == -1) & (creatinine['%s'%creatinine.columns[j]] != -1)), '%s'%creatinine.columns[i]] = creatinine['%s'%creatinine.columns[j]]
            j += 1
    elif i < len(creatinine.columns) - 1:
        creatinine.loc[((creatinine['%s'%creatinine.columns[i]] == -1) & (creatinine['%s'%creatinine.columns[i + 1]] != -1)), '%s'%creatinine.columns[i]] = (creatinine['%s'%creatinine.columns[i - 1]] + creatinine['%s'%creatinine.columns[i + 1]])/2
        creatinine.loc[(creatinine['%s'%creatinine.columns[i]] == -1), '%s'%creatinine.columns[i]] = creatinine['%s'%creatinine.columns[i - 1]]
    else:
        creatinine.loc[(creatinine['%s'%creatinine.columns[i]] == -1), '%s'%creatinine.columns[i]] = creatinine['%s'%creatinine.columns[i - 1]]
    i += 1

aki_creatinine = creatinine
print('')
for i in creatinine.columns:
    print("%s Missing Rate:"%i,1 - creatinine[creatinine['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,creatinine[creatinine['%s'%i] == -1]['%s'%i].count())
    print('')

creatinine


creatinine max columns:10
creatinine avg columns:1.573415952858271
creatinine median columns:1

creatinine Missing Rate: 0.18220749426459337
creatinine Null Count: 420

creatinine_1 Missing Rate: 0.18455263828702528
creatinine_1 Null Count: 466

creatinine_2 Missing Rate: 0.18455263828702528
creatinine_2 Null Count: 466

creatinine_3 Missing Rate: 0.18455263828702528
creatinine_3 Null Count: 466

creatinine_4 Missing Rate: 0.18455263828702528
creatinine_4 Null Count: 466

creatinine_5 Missing Rate: 0.18455263828702528
creatinine_5 Null Count: 466

creatinine_6 Missing Rate: 0.18455263828702528
creatinine_6 Null Count: 466



,creatinine,creatinine_1,creatinine_2,creatinine_3,creatinine_4,creatinine_5,creatinine_6
patientunitstayid,,,,,,,
141266,1.90,1.9000,1.9000,1.9000,1.9000,1.9000,1.9000
141304,2.34,2.3400,2.3400,2.3400,2.3400,2.3400,2.0500
141314,1.67,1.6700,1.6700,1.6700,1.6700,1.6700,1.6700
141448,1.57,1.5700,1.5700,1.5700,1.5700,1.5700,1.5700
141470,2.44,2.4400,2.4400,2.4400,2.4400,2.4400,2.4400
...,...,...,...,...,...,...,...
3353190,1.39,1.3900,1.3900,1.3900,1.6450,1.9000,1.9000
3353194,1.18,1.1800,1.1850,1.1900,1.2900,1.4250,1.5600
3353197,1.66,1.6600,1.6600,1.6600,1.6600,1.6600,1.6600


In [336]:
#condition-related
cur.execute("SELECT patientunitstayid,cirrhosis,gender,hepaticFailure,ventday1,diabetes FROM apachepredvar WHERE patientunitstayid IN (SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
condition = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','cirrhosis','gender','hepaticFailure','ventday1','diabetes'])
condition = condition.fillna(-1)
condition.set_index('patientunitstayid',inplace=True)
aki_condition = condition

for i in creatinine.columns:
    print("%s Missing Rate:"%i,1 - creatinine[creatinine['%s'%i] != -1]['%s'%i].count()/aki_patient)
condition


creatinine Missing Rate: 0.18220749426459337
creatinine_1 Missing Rate: 0.18455263828702528
creatinine_2 Missing Rate: 0.18455263828702528
creatinine_3 Missing Rate: 0.18455263828702528
creatinine_4 Missing Rate: 0.18455263828702528
creatinine_5 Missing Rate: 0.18455263828702528
creatinine_6 Missing Rate: 0.18455263828702528


,cirrhosis,gender,hepaticFailure,ventday1,diabetes
patientunitstayid,,,,,
141266,0,0,0,0,1
141304,0,0,0,1,0
141314,0,0,0,1,0
141448,0,0,0,0,0
141470,0,1,0,0,0
...,...,...,...,...,...
3353190,0,0,0,0,0
3353194,0,1,0,1,1
3353197,0,1,0,0,0


In [337]:
#glucose
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('glucose') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
glucose_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
glucose_res = glucose_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
glucose_size = glucose_res.groupby("patientunitstayid").size()
print("glucose max columns:" + str(max(glucose_size)) + "\n" + "glucose avg columns:" + str(float(statistics.mean(glucose_size))) + "\n" + "glucose median columns:" + str(int(statistics.median(glucose_size))))

#glucose turn continuous data from vertical to horizontal
glucose_res['cumCount'] = glucose_res.groupby(['patientunitstayid']).cumcount()
glucose_1 = glucose_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
glucose_1 = glucose_1.iloc[:,:1]

#glucose let format right
glucose_1 = pd.DataFrame(glucose_1)
glucose_1 = glucose_1.fillna(-1)
glucose_1 = pd.DataFrame(glucose_1,dtype = np.float)
column = ['glucose']
glucose_1.columns = column

i = 0
NextOffset = 0
glucose_dflist = []
glucose_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('glucose') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    glucose_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'glucose_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    glucose_tdflist.append(glucose_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    glucose_res = glucose_res.groupby(['patientunitstayid']).agg({'glucose_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index() 
    glucose_dflist.append(glucose_res)
    i -= 1

glucose = pd.DataFrame()
glucose = glucose_1
for data in glucose_dflist[0:]:
    glucose = pd.merge(glucose,data,on = 'patientunitstayid',how = 'left')
glucose = glucose.set_index('patientunitstayid')
glucose = glucose.fillna(-1)
for j in glucose.columns:
    glucose.loc[((glucose['%s'%j] < 10) | (glucose['%s'%j] > 600)), '%s'%j] = -1

i = 1
while i < len(glucose.columns):
    if i == 1:
        j = 2
        while j < len(glucose.columns):
            glucose.loc[((glucose['%s'%glucose.columns[i]] == -1) & (glucose['%s'%glucose.columns[j]] != -1)), '%s'%glucose.columns[i]] = glucose['%s'%glucose.columns[j]]
            j += 1
    elif i < len(glucose.columns) - 1:
        glucose.loc[((glucose['%s'%glucose.columns[i]] == -1) & (glucose['%s'%glucose.columns[i + 1]] != -1)), '%s'%glucose.columns[i]] = (glucose['%s'%glucose.columns[i - 1]] + glucose['%s'%glucose.columns[i + 1]])/2
        glucose.loc[(glucose['%s'%glucose.columns[i]] == -1), '%s'%glucose.columns[i]] = glucose['%s'%glucose.columns[i - 1]]
    else:
        glucose.loc[(glucose['%s'%glucose.columns[i]] == -1), '%s'%glucose.columns[i]] = glucose['%s'%glucose.columns[i - 1]]
    i += 1

aki_glucose = glucose
print('')
for i in glucose.columns:
    print("%s Missing Rate:"%i,1 - glucose[glucose['%s'%i] != -1]['%s'%i].count()/aki_patient)
    print("%s Null Count:"%i,glucose[glucose['%s'%i] == -1]['%s'%i].count())
    print('')

glucose


glucose max columns:25
glucose avg columns:1.6434649525558616
glucose median columns:1

glucose Missing Rate: 0.19408615855212852
glucose Null Count: 527

glucose_1 Missing Rate: 0.18883507519755294
glucose_1 Null Count: 424

glucose_2 Missing Rate: 0.18883507519755294
glucose_2 Null Count: 424

glucose_3 Missing Rate: 0.18883507519755294
glucose_3 Null Count: 424

glucose_4 Missing Rate: 0.18883507519755294
glucose_4 Null Count: 424

glucose_5 Missing Rate: 0.18883507519755294
glucose_5 Null Count: 424

glucose_6 Missing Rate: 0.18883507519755294
glucose_6 Null Count: 424



,glucose,glucose_1,glucose_2,glucose_3,glucose_4,glucose_5,glucose_6
patientunitstayid,,,,,,,
141266,423.0,423.0000,423.0000,423.0000,423.0000,423.0000,423.0000
141304,173.0,173.0000,173.0000,173.0000,173.0000,173.0000,129.0000
141314,153.0,153.0000,153.0000,153.0000,153.0000,153.0000,153.0000
141448,232.0,232.0000,232.0000,232.0000,232.0000,232.0000,232.0000
141470,153.0,153.0000,153.0000,153.0000,153.0000,153.0000,153.0000
...,...,...,...,...,...,...,...
3353190,84.0,84.0000,84.0000,84.0000,76.5000,69.0000,69.0000
3353194,145.0,145.0000,132.5000,120.0000,123.0000,131.5000,140.0000
3353197,264.0,264.0000,264.0000,264.0000,264.0000,264.0000,264.0000


In [338]:
sql="SELECT T.patientunitstayid,T.treatmentoffset,T.treatmentstring FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,treatment T WHERE TB.patientunitstayid=T.patientunitstayid AND treatmentstring LIKE '%renal|dialysis%' AND T.treatmentoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,treatmentoffset"
cur.execute(sql)
dialysis_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','treatmentoffset','treatmentstring'])
dialysis_res = dialysis_res.groupby(['patientunitstayid']).agg({'treatmentoffset':np.min}).reset_index()
dialysis_res['treatmentoffset'] = 1
dialysis_res=dialysis_res.rename(columns={'treatmentoffset':'IsDialysis'})
dialysis_res=pd.merge(akiDiagdf['patientunitstayid'],dialysis_res,on='patientunitstayid',how='left')
dialysis_res=dialysis_res.fillna(0)
dialysis = dialysis_res
dialysis.set_index('patientunitstayid',inplace=True)
aki_dialysis = dialysis
print(dialysis)


                   IsDialysis
patientunitstayid            
141266                    0.0
141304                    0.0
141314                    0.0
141448                    0.0
141470                    0.0
...                       ...
3353194                   0.0
3353197                   0.0
3353199                   0.0
3353251                   0.0
3353254                   0.0

[19615 rows x 1 columns]


In [339]:
#show how to concat two dataframe,use for loop to connect all dataframe(must start with age_res,since its missing rate is 0)
AKIPT = pd.DataFrame()
AKIPT = pd.merge(aki_age,aki_nettotal,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_dialysis,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_potassium,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_sodium,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_Hct,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_BUN,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_pH,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_OI,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_temperature,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_urine,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_MAP,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_creatinine,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_glucose,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_albumin,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_MSI,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_weight,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_condition,on='patientunitstayid',how='left')
AKIPT['aki'] = 1
AKIPT=AKIPT.fillna(-1)
AKIPT = AKIPT.loc[AKIPT['IsDialysis'] != 1]
AKIPT.drop('IsDialysis',axis = 1,inplace = True)
AKIPT.to_csv('AKIPT_RNN_6.csv', encoding='utf-8')
AKIPT

,age,nettotal,potassium,potassium_1,potassium_2,potassium_3,potassium_4,potassium_5,potassium_6,sodium,...,MSI_4,MSI_5,MSI_6,weight,cirrhosis,gender,hepaticFailure,ventday1,diabetes,aki
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141266,73.0,63.22,5.2,5.2000,5.2000,5.2000,5.2000,5.2000,5.2000,135.0,...,0.810345,0.910891,0.960000,120.40,0.0,0.0,0.0,0.0,1.0,1
141304,70.0,1253.00,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,5.8000,141.0,...,0.854015,0.854015,0.991597,-1.00,0.0,0.0,0.0,1.0,0.0,1
141314,45.0,150.00,8.0,8.0000,8.0000,8.0000,8.0000,8.0000,8.0000,158.0,...,1.545455,1.545455,1.545455,-1.00,0.0,0.0,0.0,1.0,0.0,1
141448,65.0,0.00,2.8,2.8000,2.8000,2.8000,2.8000,2.8000,2.8000,138.0,...,0.860927,0.860927,0.851351,79.80,0.0,0.0,0.0,0.0,0.0,1
141470,58.0,0.00,4.3,4.3000,4.3000,4.3000,4.3000,4.3000,4.3000,131.0,...,0.947761,0.947761,1.120000,-1.00,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353194,51.0,-302.00,3.4,3.4000,3.3500,3.3000,3.5000,3.5500,3.6000,139.0,...,1.125000,0.990385,1.147727,63.05,0.0,1.0,0.0,1.0,1.0,1
3353197,66.0,2404.60,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,4.8000,132.0,...,0.706897,0.627273,0.673077,71.50,0.0,1.0,0.0,0.0,0.0,1
3353199,66.0,17.44,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,71.50,-1.0,-1.0,-1.0,-1.0,-1.0,1


In [340]:
#get NOAKI patient's ID and diagnosisoffset
cur.execute("SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid")
akiDiagdf = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','diagnosisoffset'])


#net fluid balance
cur = conn.cursor()
cur.execute("SELECT IO.patientunitstayid,IO.intakeoutputoffset,IO.nettotal FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,intakeoutput IO WHERE TB.patientunitstayid=IO.patientunitstayid AND IO.nettotal IS NOT NULL AND IO.intakeoutputoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY IO.patientunitstayid,IO.intakeoutputoffset")
IO_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','intakeoutputoffset','nettotal'])
IO_res=IO_res.groupby(['patientunitstayid','intakeoutputoffset']).agg({'nettotal':np.max}).reset_index()
IO_res=IO_res.groupby(['patientunitstayid']).agg({'nettotal':np.sum}).reset_index()  
nettotal = IO_res
nettotal.columns = ['patientunitstayid','nettotal']
nettotal.set_index('patientunitstayid',inplace=True)
nettotal=nettotal.fillna(-1)
nettotal = pd.DataFrame(nettotal,dtype = np.float)
for i in nettotal[(nettotal['nettotal'] != -1) & (nettotal['nettotal'] > 3000)].index.values:
    nettotal.loc[i]['nettotal'] = -1

print("nettotal Missing Rate:",1 - nettotal[nettotal['nettotal']!=-1]['nettotal'].count()/all_patient)
nettotal


nettotal Missing Rate: 0.45907726600604715


,nettotal
patientunitstayid,
141168,0.00
141203,0.00
141227,1830.00
141288,2351.00
141296,620.00
...,...
3353216,-2138.56
3353226,1.41
3353235,0.00


In [341]:
#potassium
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('potassium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
potassium_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
potassium_res = potassium_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
potassium_size = potassium_res.groupby("patientunitstayid").size()
print("potassium max columns:" + str(max(potassium_size)) + "\n" + "potassium avg columns:" + str(float(statistics.mean(potassium_size))) + "\n" + "potassium median columns:" + str(int(statistics.median(potassium_size))))

#potassium turn continuous data from vertical to horizontal
potassium_res['cumCount'] = potassium_res.groupby(['patientunitstayid']).cumcount()
potassium_1 = potassium_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
potassium_1 = potassium_1.iloc[:,:1]

#potassium let format right
potassium_1 = pd.DataFrame(potassium_1)
potassium_1 = potassium_1.fillna(-1)
potassium_1 = pd.DataFrame(potassium_1,dtype = np.float)
column = ['potassium']
potassium_1.columns = column

i = 0
NextOffset = 0
potassium_dflist = []
potassium_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('potassium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    potassium_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'potassium_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    potassium_tdflist.append(potassium_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    potassium_res = potassium_res.groupby(['patientunitstayid']).agg({'potassium_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    potassium_dflist.append(potassium_res)
    i -= 1

potassium = pd.DataFrame()
potassium = potassium_1
for data in potassium_dflist[0:]:
    potassium = pd.merge(potassium,data,on = 'patientunitstayid',how = 'left')
potassium = potassium.set_index('patientunitstayid')
potassium = potassium.fillna(-1)
for j in potassium.columns:
    potassium.loc[(potassium['%s'%j] > 10), '%s'%j] = -1

i = 1
while i < len(potassium.columns):
    if i == 1:
        j = 2
        while j < len(potassium.columns):
            potassium.loc[((potassium['%s'%potassium.columns[i]] == -1) & (potassium['%s'%potassium.columns[j]] != -1)), '%s'%potassium.columns[i]] = potassium['%s'%potassium.columns[j]]
            j += 1
    elif i < len(potassium.columns) - 1:
        potassium.loc[((potassium['%s'%potassium.columns[i]] == -1) & (potassium['%s'%potassium.columns[i + 1]] != -1)), '%s'%potassium.columns[i]] = (potassium['%s'%potassium.columns[i - 1]] + potassium['%s'%potassium.columns[i + 1]])/2
        potassium.loc[(potassium['%s'%potassium.columns[i]] == -1), '%s'%potassium.columns[i]] = potassium['%s'%potassium.columns[i - 1]]
    else:
        potassium.loc[(potassium['%s'%potassium.columns[i]] == -1), '%s'%potassium.columns[i]] = potassium['%s'%potassium.columns[i - 1]]
    i += 1

print('')
for i in potassium.columns:
    print("%s Missing Rate:"%i,1 - potassium[potassium['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,potassium[potassium['%s'%i] == -1]['%s'%i].count())
    print('')

potassium

potassium max columns:31
potassium avg columns:1.5393696523340192
potassium median columns:1

potassium Missing Rate: 0.38870803999028936
potassium Null Count: 3

potassium_1 Missing Rate: 0.38869148771821416
potassium_1 Null Count: 0

potassium_2 Missing Rate: 0.38869148771821416
potassium_2 Null Count: 0

potassium_3 Missing Rate: 0.38869148771821416
potassium_3 Null Count: 0

potassium_4 Missing Rate: 0.38869148771821416
potassium_4 Null Count: 0

potassium_5 Missing Rate: 0.38869148771821416
potassium_5 Null Count: 0

potassium_6 Missing Rate: 0.38869148771821416
potassium_6 Null Count: 0



,potassium,potassium_1,potassium_2,potassium_3,potassium_4,potassium_5,potassium_6
patientunitstayid,,,,,,,
141203,3.6,3.6000,3.6000,3.6000,3.6000,3.9000,4.2000
141227,4.2,4.2000,4.2000,4.2000,4.2000,4.2000,4.2000
141229,4.0,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000
141288,3.7,3.7000,3.7000,3.7000,3.7000,3.7000,3.7000
141296,5.4,5.4000,5.4000,5.4000,5.4000,5.4000,5.4000
...,...,...,...,...,...,...,...
3353213,3.8,3.8000,3.8000,3.8000,3.8000,3.8000,3.8000
3353216,5.1,5.1000,5.1000,5.1000,5.1000,5.1000,5.1000
3353226,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,4.8000


In [342]:
#sodium
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('sodium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
sodium_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
sodium_res = sodium_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
sodium_size = sodium_res.groupby("patientunitstayid").size()
print("sodium max columns:" + str(max(sodium_size)) + "\n" + "sodium avg columns:" + str(float(statistics.mean(sodium_size))) + "\n" + "sodium median columns:" + str(int(statistics.median(sodium_size))))

#sodium turn continuous data from vertical to horizontal
sodium_res['cumCount'] = sodium_res.groupby(['patientunitstayid']).cumcount()
sodium_1 = sodium_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
sodium_1 = sodium_1.iloc[:,:1]

#sodium let format right
sodium_1 = pd.DataFrame(sodium_1)
sodium_1 = sodium_1.fillna(-1)
sodium_1 = pd.DataFrame(sodium_1,dtype = np.float)
column = ['sodium']
sodium_1.columns = column

i = 0
NextOffset = 0
sodium_dflist = []
sodium_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('sodium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    sodium_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'sodium_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    sodium_tdflist.append(sodium_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    sodium_res = sodium_res.groupby(['patientunitstayid']).agg({'sodium_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    sodium_dflist.append(sodium_res)
    i -= 1

sodium = pd.DataFrame()
sodium = sodium_1
for data in sodium_dflist[0:]:
    sodium = pd.merge(sodium,data,on = 'patientunitstayid',how = 'left')
sodium = sodium.set_index('patientunitstayid')
sodium = sodium.fillna(-1)
for j in sodium.columns:
    sodium.loc[((sodium['%s'%j] > 180) | (sodium['%s'%j] < 100)), '%s'%j] = -1

i = 1
while i < len(sodium.columns):
    if i == 1:
        j = 2
        while j < len(sodium.columns):
            sodium.loc[((sodium['%s'%sodium.columns[i]] == -1) & (sodium['%s'%sodium.columns[j]] != -1)), '%s'%sodium.columns[i]] = sodium['%s'%sodium.columns[j]]
            j += 1
    elif i < len(sodium.columns) - 1:
        sodium.loc[((sodium['%s'%sodium.columns[i]] == -1) & (sodium['%s'%sodium.columns[i + 1]] != -1)), '%s'%sodium.columns[i]] = (sodium['%s'%sodium.columns[i - 1]] + sodium['%s'%sodium.columns[i + 1]])/2
        sodium.loc[(sodium['%s'%sodium.columns[i]] == -1), '%s'%sodium.columns[i]] = sodium['%s'%sodium.columns[i - 1]]
    else:
        sodium.loc[(sodium['%s'%sodium.columns[i]] == -1), '%s'%sodium.columns[i]] = sodium['%s'%sodium.columns[i - 1]]
    i += 1

print('')
for i in sodium.columns:
    print("%s Missing Rate:"%i,1 - sodium[sodium['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,sodium[sodium['%s'%i] == -1]['%s'%i].count())
    print('')

sodium


sodium max columns:31
sodium avg columns:1.5123886023045612
sodium median columns:1

sodium Missing Rate: 0.3914943391229503
sodium Null Count: 15

sodium_1 Missing Rate: 0.39147226942685
sodium_1 Null Count: 11

sodium_2 Missing Rate: 0.39147226942685
sodium_2 Null Count: 11

sodium_3 Missing Rate: 0.39147226942685
sodium_3 Null Count: 11

sodium_4 Missing Rate: 0.39147226942685
sodium_4 Null Count: 11

sodium_5 Missing Rate: 0.39147226942685
sodium_5 Null Count: 11

sodium_6 Missing Rate: 0.39147226942685
sodium_6 Null Count: 11



,sodium,sodium_1,sodium_2,sodium_3,sodium_4,sodium_5,sodium_6
patientunitstayid,,,,,,,
141203,144.0,144.0000,144.0000,144.0000,144.0000,144.5000,145.0000
141227,133.0,133.0000,133.0000,133.0000,133.0000,133.0000,133.0000
141229,138.0,138.0000,138.0000,138.0000,138.0000,138.0000,138.0000
141288,142.0,142.0000,142.0000,142.0000,142.0000,142.0000,142.0000
141296,138.0,138.0000,138.0000,138.0000,138.0000,138.0000,138.0000
...,...,...,...,...,...,...,...
3353213,143.0,143.0000,143.0000,143.0000,143.0000,143.0000,143.0000
3353216,143.0,143.0000,143.0000,143.0000,143.0000,143.0000,143.0000
3353226,136.0,136.0000,136.0000,136.0000,136.0000,136.0000,136.0000


In [343]:
#Hct
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('Hct') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
Hct_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
Hct_res = Hct_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
Hct_size = Hct_res.groupby("patientunitstayid").size()
print("Hct max columns:" + str(max(Hct_size)) + "\n" + "Hct avg columns:" + str(float(statistics.mean(Hct_size))) + "\n" + "Hct median columns:" + str(int(statistics.median(Hct_size))))

#Hct turn continuous data from vertical to horizontal
Hct_res['cumCount'] = Hct_res.groupby(['patientunitstayid']).cumcount()
Hct_1 = Hct_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
Hct_1 = Hct_1.iloc[:,:1]

#Hct let format right
Hct_1 = pd.DataFrame(Hct_1)
Hct_1 = Hct_1.fillna(-1)
Hct_1 = pd.DataFrame(Hct_1,dtype = np.float)
column = ['Hct']
Hct_1.columns = column

i = 0
NextOffset = 0
Hct_dflist = []
Hct_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('Hct') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    Hct_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'Hct_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    Hct_tdflist.append(Hct_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    Hct_res = Hct_res.groupby(['patientunitstayid']).agg({'Hct_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    Hct_dflist.append(Hct_res)
    i -= 1

Hct = pd.DataFrame()
Hct = Hct_1
for data in Hct_dflist[0:]:
    Hct = pd.merge(Hct,data,on = 'patientunitstayid',how = 'left')
Hct = Hct.set_index('patientunitstayid')
Hct = Hct.fillna(-1)
for j in Hct.columns:
    Hct.loc[((Hct['%s'%j] > 70) | (Hct['%s'%j] < 10)), '%s'%j] = -1

i = 1
while i < len(Hct.columns):
    if i == 1:
        j = 2
        while j < len(Hct.columns):
            Hct.loc[((Hct['%s'%Hct.columns[i]] == -1) & (Hct['%s'%Hct.columns[j]] != -1)), '%s'%Hct.columns[i]] = Hct['%s'%Hct.columns[j]]
            j += 1
    elif i < len(Hct.columns) - 1:
        Hct.loc[((Hct['%s'%Hct.columns[i]] == -1) & (Hct['%s'%Hct.columns[i + 1]] != -1)), '%s'%Hct.columns[i]] = (Hct['%s'%Hct.columns[i - 1]] + Hct['%s'%Hct.columns[i + 1]])/2
        Hct.loc[(Hct['%s'%Hct.columns[i]] == -1), '%s'%Hct.columns[i]] = Hct['%s'%Hct.columns[i - 1]]
    else:
        Hct.loc[(Hct['%s'%Hct.columns[i]] == -1), '%s'%Hct.columns[i]] = Hct['%s'%Hct.columns[i - 1]]
    i += 1

print('')
for i in Hct.columns:
    print("%s Missing Rate:"%i,1 - Hct[Hct['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,Hct[Hct['%s'%i] == -1]['%s'%i].count())
    print('')


Hct


Hct max columns:32
Hct avg columns:1.444443446795902
Hct median columns:1

Hct Missing Rate: 0.3858113923771269
Hct Null Count: 55

Hct_1 Missing Rate: 0.3857231135927258
Hct_1 Null Count: 39

Hct_2 Missing Rate: 0.3857231135927258
Hct_2 Null Count: 39

Hct_3 Missing Rate: 0.3857231135927258
Hct_3 Null Count: 39

Hct_4 Missing Rate: 0.3857231135927258
Hct_4 Null Count: 39

Hct_5 Missing Rate: 0.3857231135927258
Hct_5 Null Count: 39

Hct_6 Missing Rate: 0.3857231135927258
Hct_6 Null Count: 39



,Hct,Hct_1,Hct_2,Hct_3,Hct_4,Hct_5,Hct_6
patientunitstayid,,,,,,,
141203,35.0,35.0000,35.0000,35.0000,35.0000,35.9500,36.9000
141227,27.0,28.6000,28.6000,28.6000,28.6000,28.6000,28.6000
141229,38.2,38.2000,38.2000,38.2000,38.2000,38.2000,38.2000
141288,43.7,43.7000,43.7000,43.7000,43.7000,43.7000,43.7000
141296,30.0,30.0000,30.0000,30.0000,30.0000,30.0000,30.0000
...,...,...,...,...,...,...,...
3353213,51.0,51.0000,51.0000,51.0000,51.0000,51.0000,51.0000
3353216,26.0,26.0000,26.0000,26.0000,26.0000,26.0000,26.0000
3353226,28.0,28.0000,28.0000,28.0000,28.0000,28.0000,28.0000


In [344]:
#BUN
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('BUN') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
BUN_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
BUN_res = BUN_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
BUN_size = BUN_res.groupby("patientunitstayid").size()
print("BUN max columns:" + str(max(BUN_size)) + "\n" + "BUN avg columns:" + str(float(statistics.mean(BUN_size))) + "\n" + "BUN median columns:" + str(int(statistics.median(BUN_size))))

#BUN turn continuous data from vertical to horizontal
BUN_res['cumCount'] = BUN_res.groupby(['patientunitstayid']).cumcount()
BUN_1 = BUN_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
BUN_1 = BUN_1.iloc[:,:1]

#BUN let format right
BUN_1 = pd.DataFrame(BUN_1)
BUN_1 = BUN_1.fillna(-1)
BUN_1 = pd.DataFrame(BUN_1,dtype = np.float)
column = ['BUN']
BUN_1.columns = column

i = 0
NextOffset = 0
BUN_dflist = []
BUN_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('BUN') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    BUN_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'BUN_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    BUN_tdflist.append(BUN_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    BUN_res = BUN_res.groupby(['patientunitstayid']).agg({'BUN_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    BUN_dflist.append(BUN_res)
    i -= 1

BUN = pd.DataFrame()
BUN = BUN_1
for data in BUN_dflist[0:]:
    BUN = pd.merge(BUN,data,on = 'patientunitstayid',how = 'left')
BUN = BUN.set_index('patientunitstayid')
BUN = BUN.fillna(-1)
for j in BUN.columns:
    BUN.loc[((BUN['%s'%j] < 1) | (BUN['%s'%j] > 150)), '%s'%j] = -1

i = 1
while i < len(BUN.columns):
    if i == 1:
        j = 2
        while j < len(BUN.columns):
            BUN.loc[((BUN['%s'%BUN.columns[i]] == -1) & (BUN['%s'%BUN.columns[j]] != -1)), '%s'%BUN.columns[i]] = BUN['%s'%BUN.columns[j]]
            j += 1
    elif i < len(BUN.columns) - 1:
        BUN.loc[((BUN['%s'%BUN.columns[i]] == -1) & (BUN['%s'%BUN.columns[i + 1]] != -1)), '%s'%BUN.columns[i]] = (BUN['%s'%BUN.columns[i - 1]] + BUN['%s'%BUN.columns[i + 1]])/2
        BUN.loc[(BUN['%s'%BUN.columns[i]] == -1), '%s'%BUN.columns[i]] = BUN['%s'%BUN.columns[i - 1]]
    else:
        BUN.loc[(BUN['%s'%BUN.columns[i]] == -1), '%s'%BUN.columns[i]] = BUN['%s'%BUN.columns[i - 1]]
    i += 1

print('')
for i in BUN.columns:
    print("%s Missing Rate:"%i,1 - BUN[BUN['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,BUN[BUN['%s'%i] == -1]['%s'%i].count())
    print('')


BUN


BUN max columns:11
BUN avg columns:1.2352699612674414
BUN median columns:1

BUN Missing Rate: 0.4094038975083313
BUN Null Count: 103

BUN_1 Missing Rate: 0.4093321709960054
BUN_1 Null Count: 90

BUN_2 Missing Rate: 0.4093321709960054
BUN_2 Null Count: 90

BUN_3 Missing Rate: 0.4093321709960054
BUN_3 Null Count: 90

BUN_4 Missing Rate: 0.4093321709960054
BUN_4 Null Count: 90

BUN_5 Missing Rate: 0.4093321709960054
BUN_5 Null Count: 90

BUN_6 Missing Rate: 0.4093321709960054
BUN_6 Null Count: 90



,BUN,BUN_1,BUN_2,BUN_3,BUN_4,BUN_5,BUN_6
patientunitstayid,,,,,,,
141203,9.0,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000
141227,26.0,26.0000,26.0000,26.0000,26.0000,26.0000,26.0000
141229,25.0,25.0000,25.0000,25.0000,25.0000,25.0000,25.0000
141288,17.0,17.0000,17.0000,17.0000,17.0000,17.0000,17.0000
141296,49.0,49.0000,49.0000,49.0000,49.0000,49.0000,49.0000
...,...,...,...,...,...,...,...
3353213,9.0,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000
3353216,7.0,7.0000,7.0000,7.0000,7.0000,7.0000,7.0000
3353226,42.0,42.0000,42.0000,42.0000,42.0000,42.0000,42.0000


In [345]:
#pH
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('pH') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
pH_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
pH_res = pH_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
pH_size = pH_res.groupby("patientunitstayid").size()
print("pH max columns:" + str(max(pH_size)) + "\n" + "pH avg columns:" + str(float(statistics.mean(pH_size))) + "\n" + "pH median columns:" + str(int(statistics.median(pH_size))))

#pH turn continuous data from vertical to horizontal
pH_res['cumCount'] = pH_res.groupby(['patientunitstayid']).cumcount()
pH_1 = pH_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
pH_1 = pH_1.iloc[:,:1]

#pH let format right
pH_1 = pd.DataFrame(pH_1)
pH_1 = pH_1.fillna(-1)
pH_1 = pd.DataFrame(pH_1,dtype = np.float)
column = ['pH']
pH_1.columns = column

i = 0
NextOffset = 0
pH_dflist = []
pH_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('pH') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    pH_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'pH_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    pH_tdflist.append(pH_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    pH_res = pH_res.groupby(['patientunitstayid']).agg({'pH_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    pH_dflist.append(pH_res)
    i -= 1

pH = pd.DataFrame()
pH = pH_1
for data in pH_dflist[0:]:
    pH = pd.merge(pH,data,on = 'patientunitstayid',how = 'left')
pH = pH.set_index('patientunitstayid')
pH = pH.fillna(-1)
for j in pH.columns:
    pH.loc[((pH['%s'%j] > 7.8) | (pH['%s'%j] < 6.8)), '%s'%j] = -1

i = 1
while i < len(pH.columns):
    if i == 1:
        j = 2
        while j < len(pH.columns):
            pH.loc[((pH['%s'%pH.columns[i]] == -1) & (pH['%s'%pH.columns[j]] != -1)), '%s'%pH.columns[i]] = pH['%s'%pH.columns[j]]
            j += 1
    elif i < len(pH.columns) - 1:
        pH.loc[((pH['%s'%pH.columns[i]] == -1) & (pH['%s'%pH.columns[i + 1]] != -1)), '%s'%pH.columns[i]] = (pH['%s'%pH.columns[i - 1]] + pH['%s'%pH.columns[i + 1]])/2
        pH.loc[(pH['%s'%pH.columns[i]] == -1), '%s'%pH.columns[i]] = pH['%s'%pH.columns[i - 1]]
    else:
        pH.loc[(pH['%s'%pH.columns[i]] == -1), '%s'%pH.columns[i]] = pH['%s'%pH.columns[i - 1]]
    i += 1

print('')
for i in pH.columns:
    print("%s Missing Rate:"%i,1 - pH[pH['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,pH[pH['%s'%i] == -1]['%s'%i].count())
    print('')


pH



pH max columns:30
pH avg columns:1.992710949132203
pH median columns:1

pH Missing Rate: 0.7521848999139282
pH Null Count: 84

pH_1 Missing Rate: 0.7519255809847498
pH_1 Null Count: 37

pH_2 Missing Rate: 0.7519255809847498
pH_2 Null Count: 37

pH_3 Missing Rate: 0.7519255809847498
pH_3 Null Count: 37

pH_4 Missing Rate: 0.7519255809847498
pH_4 Null Count: 37

pH_5 Missing Rate: 0.7519255809847498
pH_5 Null Count: 37

pH_6 Missing Rate: 0.7519255809847498
pH_6 Null Count: 37



,pH,pH_1,pH_2,pH_3,pH_4,pH_5,pH_6
patientunitstayid,,,,,,,
141203,7.450,7.4500,7.4500,7.4500,7.4500,7.4500,7.4500
141227,7.370,7.3700,7.3700,7.3700,7.3700,7.3700,7.3700
141296,7.540,7.5400,7.5400,7.5400,7.5400,7.5400,7.5400
141297,7.420,7.4200,7.4200,7.4200,7.4200,7.4200,7.4200
141313,7.140,7.2000,7.2000,7.2000,7.2000,7.2000,7.2000
...,...,...,...,...,...,...,...
3353147,7.308,7.3080,7.3080,7.3080,7.3080,7.3080,7.3930
3353201,7.566,7.5660,7.5660,7.5660,7.5660,7.5660,7.5660
3353213,7.200,7.2000,7.2000,7.2000,7.2000,7.2000,7.3300


In [346]:
#OI
#paO2
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('paO2') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
paO2_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','paO2_labresult'])
paO2_res = paO2_res.groupby(['patientunitstayid','labresultoffset']).agg({'paO2_labresult':np.min}).reset_index()
paO2_size = paO2_res.groupby("patientunitstayid").size()
print("paO2 max columns:" + str(max(paO2_size)) + "\n" + "paO2 avg columns:" + str(sum(paO2_size)/len(paO2_size)) + "\n" + "paO2 missing rate:" + str((19615-len(paO2_size))/19615))

#FiO2
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('FiO2') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
FiO2_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','FiO2_labresult'])
FiO2_res = FiO2_res.groupby(['patientunitstayid','labresultoffset']).agg({'FiO2_labresult':np.min}).reset_index()
FiO2_size = FiO2_res.groupby("patientunitstayid").size()
print("FiO2 max columns:" + str(max(FiO2_size)) + "\n" + "FiO2 avg columns:" + str(sum(FiO2_size)/len(FiO2_size)) + "\n" + "FiO2 missing rate:" + str((19615-len(FiO2_size))/19615))

#pick FiO2 and paO2 at the same time,can use this dataframe to calculate ratio
OI = pd.DataFrame()
OI = pd.merge(paO2_res,FiO2_res,on = ['patientunitstayid','labresultoffset'],how = 'inner')
OI_size = OI.groupby("patientunitstayid").size()
print("paO2/FiO2 max columns:" + str(max(OI_size)) + "\n" + "paO2/FiO2 avg columns:" + str(sum(OI_size)/len(OI_size)) + "\n" + "paO2/FiO2 missing rate:" + str((19615-len(OI_size))/19615))

length_OI = int(statistics.median(OI_size))

i = 0
clean_z = OI['FiO2_labresult'].fillna(-1)
clean_z[clean_z == ''] = -1
OI['FiO2_labresult'] = clean_z
clean_z = OI['FiO2_labresult'].fillna(-1)
clean_z[clean_z ==  0] = -1
OI['FiO2_labresult'] = clean_z
OI['OI'] = OI['paO2_labresult'] / OI['FiO2_labresult']
OI.drop(columns = ['paO2_labresult','FiO2_labresult'], inplace = True)

#OI turn continuous data from vertical to horizontal
OI['cumCount'] = OI.groupby(['patientunitstayid']).cumcount()
OI = OI.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'OI')
OI = OI.iloc[:,:length_OI]

#OI let format right
column = []
i = 0
while i < length_OI:
    column.append("OI_"+"%s"%str(i))
    i += 1
OI.columns = column
OI = OI.fillna(-1)
OI = pd.DataFrame(OI,dtype = np.float)

for j in OI.columns:
    OI.loc[((OI['%s'%j] < 10) & (OI['%s'%j] > 0)), '%s'%j] *= 100


print("OI 0 Missing Rate:",1 - OI[OI['OI_0']!=-1]['OI_0'].count()/all_patient)
OI



paO2 max columns:31
paO2 avg columns:1.9753754915232387
paO2 missing rate:-1.3726229926076983
FiO2 max columns:23
FiO2 avg columns:1.470459320846795
FiO2 missing rate:-1.2564873821055316
paO2/FiO2 max columns:23
paO2/FiO2 avg columns:1.4881642936661392
paO2/FiO2 missing rate:-0.7401988274279888
OI 0 Missing Rate: 0.8133952020480678


,OI_0
patientunitstayid,
141203,51.000000
141227,177.777778
141296,257.142857
141297,118.000000
141313,92.000000
...,...
3353147,205.000000
3353201,329.000000
3353213,-1.000000


In [347]:
#temperature
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartvalue IS NOT NULL AND N.nursingchartcelltypevalname IN('Temperature (C)') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
temperature_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','nursingchartoffset','nursingchartvalue'])
temperature_res = temperature_res.groupby(['patientunitstayid','nursingchartoffset']).agg({'nursingchartvalue':np.max}).reset_index()
temperature_size = temperature_res.groupby("patientunitstayid").size()
print("temperature max columns:" + str(max(temperature_size)) + "\n" + "temperature avg columns:" + str(float(statistics.mean(temperature_size))) + "\n" + "temperature median columns:" + str(int(statistics.median(temperature_size))))

#temperature turn continuous data from vertical to horizontal
temperature_res['cumCount'] = temperature_res.groupby(['patientunitstayid']).cumcount()
temperature_1 = temperature_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartvalue')
temperature_1 = temperature_1.iloc[:,:1]

#temperature let format right
temperature_1 = pd.DataFrame(temperature_1)
temperature_1 = temperature_1.fillna(-1)
temperature_1 = pd.DataFrame(temperature_1,dtype = np.float)
column = ['temperature']
temperature_1.columns = column

i = 0
NextOffset = 0
temperature_dflist = []
temperature_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartvalue IS NOT NULL AND N.nursingchartcelltypevalname IN('Temperature (C)') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset -"+str(PreOffset)+" ORDER BY N.patientunitstayid,N.nursingchartoffset"
    cur.execute(sql)
    temperature_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','nursingchartoffset'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'temperature_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    temperature_tdflist.append(temperature_res.groupby(['patientunitstayid']).agg({'nursingchartoffset'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    temperature_res = temperature_res.groupby(['patientunitstayid']).agg({'temperature_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    temperature_dflist.append(temperature_res)
    i -= 1

temperature = pd.DataFrame()
temperature = temperature_1
for data in temperature_dflist[0:]:
    temperature = pd.merge(temperature,data,on = 'patientunitstayid',how = 'left')
temperature = temperature.set_index('patientunitstayid')
temperature = pd.DataFrame(temperature,dtype = np.float)
temperature = temperature.fillna(-1)
for j in temperature.columns:
    temperature.loc[((temperature['%s'%j] < 33) | (temperature['%s'%j] > 42)), '%s'%j] = -1

i = 1
while i < len(temperature.columns):
    if i == 1:
        j = 2
        while j < len(temperature.columns):
            temperature.loc[((temperature['%s'%temperature.columns[i]] == -1) & (temperature['%s'%temperature.columns[j]] != -1)), '%s'%temperature.columns[i]] = temperature['%s'%temperature.columns[j]]
            j += 1
    elif i < len(temperature.columns) - 1:
        temperature.loc[((temperature['%s'%temperature.columns[i]] == -1) & (temperature['%s'%temperature.columns[i + 1]] != -1)), '%s'%temperature.columns[i]] = (temperature['%s'%temperature.columns[i - 1]] + temperature['%s'%temperature.columns[i + 1]])/2
        temperature.loc[(temperature['%s'%temperature.columns[i]] == -1), '%s'%temperature.columns[i]] = temperature['%s'%temperature.columns[i - 1]]
    else:
        temperature.loc[(temperature['%s'%temperature.columns[i]] == -1), '%s'%temperature.columns[i]] = temperature['%s'%temperature.columns[i - 1]]
    i += 1

print('')
for i in temperature.columns:
    print("%s Missing Rate:"%i,1 - temperature[temperature['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,temperature[temperature['%s'%i] == -1]['%s'%i].count())
    print('')


temperature

temperature max columns:138
temperature avg columns:2.750843088297286
temperature median columns:2

temperature Missing Rate: 0.32375140694312643
temperature Null Count: 790

temperature_1 Missing Rate: 0.3223554986647834
temperature_1 Null Count: 537

temperature_2 Missing Rate: 0.3223554986647834
temperature_2 Null Count: 537

temperature_3 Missing Rate: 0.3223554986647834
temperature_3 Null Count: 537

temperature_4 Missing Rate: 0.3223554986647834
temperature_4 Null Count: 537

temperature_5 Missing Rate: 0.3223554986647834
temperature_5 Null Count: 537

temperature_6 Missing Rate: 0.3223554986647834
temperature_6 Null Count: 537



,temperature,temperature_1,temperature_2,temperature_3,temperature_4,temperature_5,temperature_6
patientunitstayid,,,,,,,
141203,36.4,36.4,36.7,36.5,36.3,36.3,36.3
141227,37.6,37.6,37.4,37.2,37.8,38.4,37.7
141229,36.4,36.4,36.4,36.4,36.4,36.4,36.4
141284,36.6,36.6,36.6,36.6,36.6,36.6,36.6
141288,37.5,37.5,37.5,37.5,37.5,37.5,37.5
...,...,...,...,...,...,...,...
3353216,34.9,34.9,34.9,34.9,34.9,34.9,34.9
3353226,36.7,36.7,36.7,36.7,36.7,36.7,36.7
3353235,36.6,36.6,36.6,36.6,36.6,36.6,36.6


In [361]:
#urine
i = 0
NextOffset = 0
urine_dflist = []
for i in range(1,1 + int(int(strsql_timeoffst)/offset)):
    PreOffset = NextOffset
    NextOffset = offset*i
    sql = "SELECT IO.patientunitstayid,IO.intakeoutputoffset,IO.cellvaluenumeric FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,intakeoutput IO WHERE TB.patientunitstayid=IO.patientunitstayid AND IO.celllabel IN('Urine') AND IO.intakeoutputoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY IO.patientunitstayid,IO.intakeoutputoffset"
    cur.execute(sql)
    urine_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','intakeoutputoffset_'+str(i),'urine_'+str(i)])
    urine_tdflist.append(urine_res.groupby(['patientunitstayid']).agg({'intakeoutputoffset_'+str(i):np.sum}).reset_index())
    urine_res = urine_res.groupby(['patientunitstayid']).agg({'urine_'+str(i):np.sum}).reset_index() 
    urine_dflist.append(urine_res)


urine = pd.DataFrame()
urine = urine_dflist[0]
for data in urine_dflist[1:]:
    urine = pd.merge(urine,data,on = 'patientunitstayid',how = 'left')
urine = urine.set_index('patientunitstayid')
urine = urine.fillna(-1)
# print(len(urine.index))
# i = 0
# total = []
# while i < len(urine.index):
#     a = 0
#     if urine.iloc[i]['urine_1'] != -1:
#         a += float(urine.iloc[i]['urine_1'])
#     if urine.iloc[i]['urine_2'] != -1:
#         a += float(urine.iloc[i]['urine_2'])
#     if urine.iloc[i]['urine_3'] != -1:
#         a += float(urine.iloc[i]['urine_3'])
#     if urine.iloc[i]['urine_4'] != -1:
#         a += float(urine.iloc[i]['urine_4'])
#     total.append(a)
#     i += 1
# urine['urine_total'] = total
urine = pd.DataFrame(urine,dtype = np.float)

for i in temperature.columns:
    print("%s Missing Rate:"%i,1 - temperature[temperature['%s'%i] != -1]['%s'%i].count()/all_patient)

urine



temperature Missing Rate: 0.32375140694312643
temperature_1 Missing Rate: 0.3223554986647834
temperature_2 Missing Rate: 0.3223554986647834
temperature_3 Missing Rate: 0.3223554986647834
temperature_4 Missing Rate: 0.3223554986647834
temperature_5 Missing Rate: 0.3223554986647834
temperature_6 Missing Rate: 0.3223554986647834


,urine_1,urine_2,urine_3,urine_4,urine_5,urine_6
patientunitstayid,,,,,,
141392,100.0,-1.0,-1.0,-1.0,-1.0,-1.0
141631,400.0,-1.0,-1.0,-1.0,-1.0,-1.0
141800,850.0,-1.0,200.0,-1.0,500.0,-1.0
141822,500.0,-1.0,-1.0,-1.0,-1.0,-1.0
141939,50.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...
3353198,50.0,-1.0,-1.0,-1.0,-1.0,-1.0
3353201,725.0,-1.0,-1.0,-1.0,-1.0,-1.0
3353213,1600.0,400.0,-1.0,-1.0,-1.0,-1.0


In [362]:
#MAP
cur.execute("SELECT VP.patientunitstayid,VP.observationoffset,VP.noninvasivediastolic,VP.noninvasivesystolic FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,vitalaperiodic VP WHERE TB.patientunitstayid=VP.patientunitstayid AND noninvasivesystolic IS NOT NULL AND noninvasivesystolic IS NOT NULL AND VP.observationoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,observationoffset")
MAP_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','observationoffset','diastolic','systolic'])
MAP_res = MAP_res.groupby(['patientunitstayid','observationoffset']).agg({'diastolic':np.max,'systolic':np.max}).reset_index()
MAP_size = MAP_res.groupby("patientunitstayid").size()
print("MAP max columns:" + str(max(MAP_size)) + "\n" + "MAP avg columns:" + str(float(statistics.mean(MAP_size))) + "\n" + "MAP median columns:" + str(int(statistics.median(MAP_size))))

#MAP turn continuous data from vertical to horizontal
MAP_res['cumCount'] = MAP_res.groupby(['patientunitstayid']).cumcount()
MAP_1 = MAP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'systolic')
MAP_1 = MAP_1.iloc[:,:1]

#MAP let format right
MAP_1 = pd.DataFrame(MAP_1)
MAP_1 = MAP_1.fillna(-1)
MAP_1 = pd.DataFrame(MAP_1,dtype = np.float)
column = ['MAP']
MAP_1.columns = column

i = 0
NextOffset = 0
MAP_dflist = []
MAP_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql =           "SELECT VP.patientunitstayid,VP.observationoffset,VP.noninvasivediastolic,VP.noninvasivesystolic FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,vitalaperiodic VP WHERE TB.patientunitstayid=VP.patientunitstayid AND noninvasivesystolic IS NOT NULL AND noninvasivesystolic IS NOT NULL AND VP.observationoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset -"+str(PreOffset)+" ORDER BY patientunitstayid,observationoffset"
    cur.execute(sql)
    MAP_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','nursingchartoffset'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'diastolic_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'systolic_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    MAP_tdflist.append(MAP_res.groupby(['patientunitstayid']).agg({'nursingchartoffset'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index())
    MAP_res = MAP_res.groupby(['patientunitstayid']).agg({'diastolic_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max,'systolic_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    MAP_dflist.append(MAP_res)
    i -= 1

MAP = pd.DataFrame()
MAP = MAP_1
for data in MAP_dflist[0:]:
    MAP = pd.merge(MAP,data,on = 'patientunitstayid',how = 'left')
MAP = MAP.set_index('patientunitstayid')
MAP = pd.DataFrame(MAP,dtype = np.float)
MAP = MAP.fillna(-1)

j = 1
while j < int(strsql_timeoffst)/offset + 1:
    MAP.loc[((MAP['diastolic_%s'%j] < 30) | (MAP['diastolic_%s'%j] > 140)), 'diastolic_%s'%j] = -1
    MAP.loc[((MAP['systolic_%s'%j] < 60) | (MAP['systolic_%s'%j] > 200)), 'systolic_%s'%j] = -1
    j += 1

i = 1
while i < int(strsql_timeoffst)/offset + 1:
    if i == 1:
        j = 2
        while j < int(strsql_timeoffst)/offset + 1:
            MAP.loc[((MAP['diastolic_%s'%i] == -1) & (MAP['diastolic_%s'%j] != -1)), 'diastolic_%s'%i] = MAP['diastolic_%s'%j]
            MAP.loc[((MAP['systolic_%s'%i] == -1) & (MAP['systolic_%s'%j] != -1)), 'systolic_%s'%i] = MAP['systolic_%s'%j]
            j += 1
    elif i < int(strsql_timeoffst)/offset:
        MAP.loc[((MAP['diastolic_%s'%i] == -1) & (MAP['diastolic_%s'%(i + 1)] != -1)), 'diastolic_%s'%i] = (MAP['diastolic_%s'%(i - 1)] + MAP['diastolic_%s'%(i + 1)])/2
        MAP.loc[(MAP['diastolic_%s'%i] == -1), 'diastolic_%s'%(i)] = MAP['diastolic_%s'%(i - 1)]
        MAP.loc[((MAP['systolic_%s'%(i)] == -1) & (MAP['systolic_%s'%(i + 1)] != -1)), 'systolic_%s'%(i)] = (MAP['systolic_%s'%(i - 1)] + MAP['systolic_%s'%(i + 1)])/2
        MAP.loc[(MAP['systolic_%s'%i] == -1), 'systolic_%s'%i] = MAP['systolic_%s'%(i - 1)]
    else:
        MAP.loc[(MAP['diastolic_%s'%(i)] == -1), 'diastolic_%s'%(i)] = MAP['diastolic_%s'%(i - 1)]
        MAP.loc[(MAP['systolic_%s'%(i)] == -1), 'systolic_%s'%(i)] = MAP['systolic_%s'%(i - 1)]
    MAP['MAP_%s'%i] = (MAP['systolic_%s'%i] - MAP['diastolic_%s'%i])/3 + MAP['diastolic_%s'%i]
    i += 1

i = 1
while i < int(strsql_timeoffst)/offset + 1:
    MAP.drop(columns = ['diastolic_%s'%i], inplace = True)
    MAP.drop(columns = ['systolic_%s'%i], inplace = True)
    i += 1

MAP.drop(columns = ['MAP'], inplace = True)

print('')
for i in MAP.columns:
    print("%s Missing Rate:"%i,1 - MAP[MAP['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,MAP[MAP['%s'%i] == -1]['%s'%i].count())
    print('')

MAP

MAP max columns:536
MAP avg columns:5.655467323601611
MAP median columns:3

MAP_1 Missing Rate: 0.37104676568603645
MAP_1 Null Count: 477

MAP_2 Missing Rate: 0.37104676568603645
MAP_2 Null Count: 477

MAP_3 Missing Rate: 0.37104676568603645
MAP_3 Null Count: 477

MAP_4 Missing Rate: 0.37104676568603645
MAP_4 Null Count: 477

MAP_5 Missing Rate: 0.37104676568603645
MAP_5 Null Count: 477

MAP_6 Missing Rate: 0.37104676568603645
MAP_6 Null Count: 477



,MAP_1,MAP_2,MAP_3,MAP_4,MAP_5,MAP_6
patientunitstayid,,,,,,
141203,87.333333,87.333333,87.333333,87.333333,87.333333,87.333333
141227,97.666667,97.666667,97.666667,97.666667,97.666667,97.666667
141229,96.333333,96.333333,96.333333,96.333333,96.333333,96.333333
141284,84.666667,84.666667,84.666667,84.666667,84.666667,84.666667
141288,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000
...,...,...,...,...,...,...
3353216,93.333333,93.333333,93.333333,93.333333,93.333333,93.333333
3353226,113.666667,113.666667,113.666667,113.666667,113.666667,113.666667
3353235,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000


In [363]:
#age,since its missing rate is 0,so I don't print its relative information
cur.execute("SELECT patientunitstayid,age FROM patient WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
age_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','age'])
age = age_res
age = np.array(age)
age = age.tolist()
for i in age:
    if i[1].isdigit() == False:
        i[1] = ''.join([x for x in i[1] if x.isdigit()])
        if i[1] != '':
            i[1] = int(i[1])
        else:
            i[1] = -1
    else:
        i[1] = int(i[1])
age = pd.DataFrame(age)
age.columns = ['patientunitstayid','age']
age.set_index('patientunitstayid',inplace=True)
age = pd.DataFrame(age,dtype = np.float)

age


,age
patientunitstayid,
141168,70.0
141178,52.0
141179,52.0
141194,68.0
141196,71.0
...,...
3353216,50.0
3353226,79.0
3353235,50.0


In [364]:
#weight
cur.execute("SELECT patientunitstayid,admissionweight FROM patient WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) AND admissionweight IS NOT NULL ORDER BY patientunitstayid")
weight_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','weight'])
weight_size = weight_res.groupby("patientunitstayid").size()
print("weight max columns:" + str(max(weight_size)) + "\n" + "weight avg columns:" + str(sum(weight_size)/len(weight_size)) + "\n" + "weight missing rate:" + str((181244-len(weight_size))/181244))

#weight turn continuous data from vertical to horizontal
weight_res['cumCount'] = weight_res.groupby(['patientunitstayid']).cumcount()
weight = weight_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'weight')
# weight = weight.iloc[:,:length_weight]

#weight let format right
weight = weight.fillna(-1)
weight = pd.DataFrame(weight,dtype = np.float)
weight.columns = ['weight']
weight.loc[((weight['weight'] < 1) | (weight['weight'] > 635)), 'weight'] = -1

weight


weight max columns:1
weight avg columns:1.0
weight missing rate:0.08788152987133367


,weight
patientunitstayid,
141168,84.3
141178,54.4
141194,73.9
141197,102.1
141203,70.2
...,...
3353216,55.4
3353226,58.4
3353235,90.0


In [365]:
#albumin
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('albumin') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
albumin_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
albumin_res = albumin_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
albumin_size = albumin_res.groupby("patientunitstayid").size()
print("albumin max columns:" + str(max(albumin_size)) + "\n" + "albumin avg columns:" + str(float(statistics.mean(albumin_size))) + "\n" + "albumin median columns:" + str(int(statistics.median(albumin_size))))

#albumin turn continuous data from vertical to horizontal
albumin_res['cumCount'] = albumin_res.groupby(['patientunitstayid']).cumcount()
albumin_1 = albumin_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
albumin_1 = albumin_1.iloc[:,:1]

#albumin let format right
albumin_1 = pd.DataFrame(albumin_1)
albumin_1 = albumin_1.fillna(-1)
albumin_1 = pd.DataFrame(albumin_1,dtype = np.float)
column = ['albumin']
albumin_1.columns = column

i = 0
NextOffset = 0
albumin_dflist = []
albumin_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('albumin') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    albumin_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'albumin_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    albumin_tdflist.append(albumin_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    albumin_res = albumin_res.groupby(['patientunitstayid']).agg({'albumin_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    albumin_dflist.append(albumin_res)
    i -= 1

albumin = pd.DataFrame()
albumin = albumin_1
for data in albumin_dflist[0:]:
    albumin = pd.merge(albumin,data,on = 'patientunitstayid',how = 'left')
albumin = albumin.set_index('patientunitstayid')
albumin = albumin.fillna(-1)
for j in albumin.columns:
    albumin.loc[((albumin['%s'%j] < 1) | (albumin['%s'%j] > 10)), '%s'%j] = -1

i = 1
while i < len(albumin.columns):
    if i == 1:
        j = 2
        while j < len(albumin.columns):
            albumin.loc[((albumin['%s'%albumin.columns[i]] == -1) & (albumin['%s'%albumin.columns[j]] != -1)), '%s'%albumin.columns[i]] = albumin['%s'%albumin.columns[j]]
            j += 1
    elif i < len(albumin.columns) - 1:
        albumin.loc[((albumin['%s'%albumin.columns[i]] == -1) & (albumin['%s'%albumin.columns[i + 1]] != -1)), '%s'%albumin.columns[i]] = (albumin['%s'%albumin.columns[i - 1]] + albumin['%s'%albumin.columns[i + 1]])/2
        albumin.loc[(albumin['%s'%albumin.columns[i]] == -1), '%s'%albumin.columns[i]] = albumin['%s'%albumin.columns[i - 1]]
    else:
        albumin.loc[(albumin['%s'%albumin.columns[i]] == -1), '%s'%albumin.columns[i]] = albumin['%s'%albumin.columns[i - 1]]
    i += 1

print('')
for i in albumin.columns:
    print("%s Missing Rate:"%i,1 - albumin[albumin['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,albumin[albumin['%s'%i] == -1]['%s'%i].count())
    print('')


albumin



albumin max columns:5
albumin avg columns:1.084604894853944
albumin median columns:1

albumin Missing Rate: 0.6159597007349209
albumin Null Count: 60

albumin_1 Missing Rate: 0.6159321136147955
albumin_1 Null Count: 55

albumin_2 Missing Rate: 0.6159321136147955
albumin_2 Null Count: 55

albumin_3 Missing Rate: 0.6159321136147955
albumin_3 Null Count: 55

albumin_4 Missing Rate: 0.6159321136147955
albumin_4 Null Count: 55

albumin_5 Missing Rate: 0.6159321136147955
albumin_5 Null Count: 55

albumin_6 Missing Rate: 0.6159321136147955
albumin_6 Null Count: 55



,albumin,albumin_1,albumin_2,albumin_3,albumin_4,albumin_5,albumin_6
patientunitstayid,,,,,,,
141203,1.4,1.4000,1.4000,1.4000,1.4000,1.4000,1.4000
141229,3.1,3.1000,3.1000,3.1000,3.1000,3.1000,3.1000
141288,3.0,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000
141296,2.1,2.1000,2.1000,2.1000,2.1000,2.1000,2.1000
141313,2.7,2.7000,2.7000,2.7000,2.7000,2.7000,2.7000
...,...,...,...,...,...,...,...
3353140,3.7,3.7000,3.7000,3.7000,3.7000,3.7000,3.7000
3353145,2.6,2.6000,2.6000,2.6000,2.6000,2.6000,2.6000
3353147,4.6,4.6000,4.6000,4.6000,4.6000,4.6000,4.6000


In [366]:
#creatinine
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('creatinine') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
creatinine_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
creatinine_res = creatinine_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
creatinine_size = creatinine_res.groupby("patientunitstayid").size()
print("creatinine max columns:" + str(max(creatinine_size)) + "\n" + "creatinine avg columns:" + str(float(statistics.mean(creatinine_size))) + "\n" + "creatinine median columns:" + str(int(statistics.median(creatinine_size))))

#creatinine turn continuous data from vertical to horizontal
creatinine_res['cumCount'] = creatinine_res.groupby(['patientunitstayid']).cumcount()
creatinine_1 = creatinine_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
creatinine_1 = creatinine_1.iloc[:,:1]

#creatinine let format right
creatinine_1 = pd.DataFrame(creatinine_1)
creatinine_1 = creatinine_1.fillna(-1)
creatinine_1 = pd.DataFrame(creatinine_1,dtype = np.float)
column = ['creatinine']
creatinine_1.columns = column

i = 0
NextOffset = 0
creatinine_dflist = []
creatinine_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('creatinine') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    creatinine_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'creatinine_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    creatinine_tdflist.append(creatinine_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.sum}).reset_index())
    creatinine_res = creatinine_res.groupby(['patientunitstayid']).agg({'creatinine_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    creatinine_dflist.append(creatinine_res)
    i -= 1

creatinine = pd.DataFrame()
creatinine = creatinine_1
for data in creatinine_dflist[0:]:
    creatinine = pd.merge(creatinine,data,on = 'patientunitstayid',how = 'left')
creatinine = creatinine.set_index('patientunitstayid')
creatinine = creatinine.fillna(-1)
for j in creatinine.columns:
    creatinine.loc[((creatinine['%s'%j] < 0.2) | (creatinine['%s'%j] > 10)), '%s'%j] = -1

i = 1
while i < len(creatinine.columns):
    if i == 1:
        j = 2
        while j < len(creatinine.columns):
            creatinine.loc[((creatinine['%s'%creatinine.columns[i]] == -1) & (creatinine['%s'%creatinine.columns[j]] != -1)), '%s'%creatinine.columns[i]] = creatinine['%s'%creatinine.columns[j]]
            j += 1
    elif i < len(creatinine.columns) - 1:
        creatinine.loc[((creatinine['%s'%creatinine.columns[i]] == -1) & (creatinine['%s'%creatinine.columns[i + 1]] != -1)), '%s'%creatinine.columns[i]] = (creatinine['%s'%creatinine.columns[i - 1]] + creatinine['%s'%creatinine.columns[i + 1]])/2
        creatinine.loc[(creatinine['%s'%creatinine.columns[i]] == -1), '%s'%creatinine.columns[i]] = creatinine['%s'%creatinine.columns[i - 1]]
    else:
        creatinine.loc[(creatinine['%s'%creatinine.columns[i]] == -1), '%s'%creatinine.columns[i]] = creatinine['%s'%creatinine.columns[i - 1]]
    i += 1

print('')
for i in creatinine.columns:
    print("%s Missing Rate:"%i,1 - creatinine[creatinine['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,creatinine[creatinine['%s'%i] == -1]['%s'%i].count())
    print('')

creatinine


creatinine max columns:11
creatinine avg columns:1.2416674412120086
creatinine median columns:1

creatinine Missing Rate: 0.4114398269735826
creatinine Null Count: 917

creatinine_1 Missing Rate: 0.4114012050054071
creatinine_1 Null Count: 910

creatinine_2 Missing Rate: 0.4114012050054071
creatinine_2 Null Count: 910

creatinine_3 Missing Rate: 0.4114012050054071
creatinine_3 Null Count: 910

creatinine_4 Missing Rate: 0.4114012050054071
creatinine_4 Null Count: 910

creatinine_5 Missing Rate: 0.4114012050054071
creatinine_5 Null Count: 910

creatinine_6 Missing Rate: 0.4114012050054071
creatinine_6 Null Count: 910



,creatinine,creatinine_1,creatinine_2,creatinine_3,creatinine_4,creatinine_5,creatinine_6
patientunitstayid,,,,,,,
141203,0.39,0.3900,0.3900,0.3900,0.3900,0.4750,0.5600
141227,1.40,1.5000,1.5000,1.5000,1.5000,1.5000,1.5000
141229,1.36,1.3600,1.3600,1.3600,1.3600,1.3600,1.3600
141288,0.95,0.9500,0.9500,0.9500,0.9500,0.9500,0.9500
141296,5.67,5.6700,5.6700,5.6700,5.6700,5.6700,5.6700
...,...,...,...,...,...,...,...
3353213,0.50,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
3353216,0.73,0.7300,0.7300,0.7300,0.7300,0.7300,0.7300
3353226,4.88,4.8800,4.8800,4.8800,4.8800,4.8800,4.8800


In [367]:
#condition-related
cur.execute("SELECT patientunitstayid,cirrhosis,gender,hepaticFailure,ventday1,diabetes FROM apachepredvar WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
condition = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','cirrhosis','gender','hepaticFailure','ventday1','diabetes'])
condition = condition.fillna(-1)
condition.set_index('patientunitstayid',inplace=True)


for i in condition.columns:
    print("%s Missing Rate:"%i,1 - condition[condition['%s'%i] != -1]['%s'%i].count()/all_patient)

condition


cirrhosis Missing Rate: 0.15796384983778777
gender Missing Rate: 0.15873628920129768
hepaticFailure Missing Rate: 0.15796384983778777
ventday1 Missing Rate: 0.15796384983778777
diabetes Missing Rate: 0.15796384983778777


,cirrhosis,gender,hepaticFailure,ventday1,diabetes
patientunitstayid,,,,,
141168,0,1,0,0,0
141178,0,1,0,0,0
141194,0,0,0,0,1
141197,0,0,0,0,0
141203,0,1,0,1,1
...,...,...,...,...,...
3353216,0,1,0,1,0
3353226,0,1,0,1,1
3353235,0,0,0,0,0


In [368]:
#glucose
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('glucose') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
glucose_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
glucose_res = glucose_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.max}).reset_index()
glucose_size = glucose_res.groupby("patientunitstayid").size()
print("glucose max columns:" + str(max(glucose_size)) + "\n" + "glucose avg columns:" + str(float(statistics.mean(glucose_size))) + "\n" + "glucose median columns:" + str(int(statistics.median(glucose_size))))

#glucose turn continuous data from vertical to horizontal
glucose_res['cumCount'] = glucose_res.groupby(['patientunitstayid']).cumcount()
glucose_1 = glucose_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
glucose_1 = glucose_1.iloc[:,:1]

#glucose let format right
glucose_1 = pd.DataFrame(glucose_1)
glucose_1 = glucose_1.fillna(-1)
glucose_1 = pd.DataFrame(glucose_1,dtype = np.float)
column = ['glucose']
glucose_1.columns = column

i = 0
NextOffset = 0
glucose_dflist = []
glucose_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labresult IS NOT NULL AND L.labname IN('glucose') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    glucose_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i),'glucose_'+str(int(int(strsql_timeoffst)/offset) + 1 - i)])
    glucose_tdflist.append(glucose_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index())
    glucose_res = glucose_res.groupby(['patientunitstayid']).agg({'glucose_'+str(int(int(strsql_timeoffst)/offset) + 1 - i):np.max}).reset_index() 
    glucose_dflist.append(glucose_res)
    i -= 1

glucose = pd.DataFrame()
glucose = glucose_1
for data in glucose_dflist[0:]:
    glucose = pd.merge(glucose,data,on = 'patientunitstayid',how = 'left')
glucose = glucose.set_index('patientunitstayid')
glucose = glucose.fillna(-1)
for j in glucose.columns:
    glucose.loc[((glucose['%s'%j] < 10) | (glucose['%s'%j] > 600)), '%s'%j] = -1

i = 1
while i < len(glucose.columns):
    if i == 1:
        j = 2
        while j < len(glucose.columns):
            glucose.loc[((glucose['%s'%glucose.columns[i]] == -1) & (glucose['%s'%glucose.columns[j]] != -1)), '%s'%glucose.columns[i]] = glucose['%s'%glucose.columns[j]]
            j += 1
    elif i < len(glucose.columns) - 1:
        glucose.loc[((glucose['%s'%glucose.columns[i]] == -1) & (glucose['%s'%glucose.columns[i + 1]] != -1)), '%s'%glucose.columns[i]] = (glucose['%s'%glucose.columns[i - 1]] + glucose['%s'%glucose.columns[i + 1]])/2
        glucose.loc[(glucose['%s'%glucose.columns[i]] == -1), '%s'%glucose.columns[i]] = glucose['%s'%glucose.columns[i - 1]]
    else:
        glucose.loc[(glucose['%s'%glucose.columns[i]] == -1), '%s'%glucose.columns[i]] = glucose['%s'%glucose.columns[i - 1]]
    i += 1

print('')
for i in glucose.columns:
    print("%s Missing Rate:"%i,1 - glucose[glucose['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,glucose[glucose['%s'%i] == -1]['%s'%i].count())
    print('')

glucose



glucose max columns:31
glucose avg columns:1.3985482970407594
glucose median columns:1

glucose Missing Rate: 0.4188883494074287
glucose Null Count: 2137

glucose_1 Missing Rate: 0.41557789499238595
glucose_1 Null Count: 1537

glucose_2 Missing Rate: 0.41557789499238595
glucose_2 Null Count: 1537

glucose_3 Missing Rate: 0.41557789499238595
glucose_3 Null Count: 1537

glucose_4 Missing Rate: 0.41557789499238595
glucose_4 Null Count: 1537

glucose_5 Missing Rate: 0.41557789499238595
glucose_5 Null Count: 1537

glucose_6 Missing Rate: 0.41557789499238595
glucose_6 Null Count: 1537



,glucose,glucose_1,glucose_2,glucose_3,glucose_4,glucose_5,glucose_6
patientunitstayid,,,,,,,
141203,95.0,95.0000,95.0000,95.0000,95.0000,120.0000,145.0000
141227,110.0,157.0000,157.0000,157.0000,157.0000,157.0000,157.0000
141229,271.0,271.0000,271.0000,271.0000,271.0000,271.0000,271.0000
141288,84.0,84.0000,84.0000,84.0000,84.0000,84.0000,84.0000
141296,95.0,95.0000,95.0000,95.0000,95.0000,95.0000,95.0000
...,...,...,...,...,...,...,...
3353213,100.0,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000
3353216,167.0,167.0000,167.0000,167.0000,167.0000,167.0000,167.0000
3353226,175.0,175.0000,175.0000,175.0000,175.0000,175.0000,175.0000


In [369]:
cur.execute("SELECT T.patientunitstayid,T.treatmentoffset,T.treatmentstring FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,treatment T WHERE TB.patientunitstayid=T.patientunitstayid AND treatmentstring LIKE '%renal|dialysis%' AND T.treatmentoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,treatmentoffset")
dialysis_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','treatmentoffset','treatmentstring'])
dialysis_res = dialysis_res.groupby(['patientunitstayid']).agg({'treatmentoffset':np.min}).reset_index()
dialysis_res['treatmentoffset'] = 1
dialysis_res = dialysis_res.rename(columns = {'treatmentoffset':'IsDialysis'})
dialysis_res = pd.merge(akiDiagdf['patientunitstayid'],dialysis_res,on = 'patientunitstayid',how = 'left')
dialysis_res = dialysis_res.fillna(0)
dialysis = dialysis_res
dialysis.set_index('patientunitstayid',inplace=True)
print(dialysis)


                   IsDialysis
patientunitstayid            
141168                    0.0
141203                    0.0
141227                    0.0
141229                    0.0
141284                    0.0
...                       ...
3353216                   0.0
3353226                   0.0
3353235                   0.0
3353237                   0.0
3353263                   0.0

[153494 rows x 1 columns]


In [370]:
#MSI modified shock index
store = []
epoch = int(int(strsql_timeoffst)/offset)
while epoch > 0:
    PreOffset = offset*(epoch - 1)
    NextOffset = offset*epoch
    cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartvalue IS NOT NULL AND N.nursingchartcelltypevalname IN('Heart Rate') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY N.patientunitstayid,N.nursingchartoffset")
    HR_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','nursingchartoffset','HR_nursingchartvalue'])
    cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartvalue IS NOT NULL AND N.nursingchartcelltypevalname IN('Non-Invasive BP Systolic') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY N.patientunitstayid,N.nursingchartoffset")
    SBP_res = pd.DataFrame(cur.fetchall(),columns =[ 'patientunitstayid','nursingchartoffset','SBP_nursingchartvalue'])

    # time offset
    HR_res['cumCount'] = HR_res.groupby(['patientunitstayid']).cumcount()
    HR_time = HR_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')

    SBP_res['cumCount'] = SBP_res.groupby(['patientunitstayid']).cumcount()
    SBP_time = SBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')


    #real value
    HR_res['cumCount'] = HR_res.groupby(['patientunitstayid']).cumcount()
    HR = HR_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'HR_nursingchartvalue')

    SBP_res['cumCount'] = SBP_res.groupby(['patientunitstayid']).cumcount()
    SBP = SBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'SBP_nursingchartvalue')


    HR = HR.fillna(-1)
    SBP = SBP.fillna(-1)
    count = 0

    i = 0
    column = []
    while i < len(HR.columns):
        column.append("HR_%s"%i)
        i += 1
    HR.columns = column

    i = 0
    column = []
    while i < len(SBP.columns):
        column.append("SBP_%s"%i)
        i += 1
    SBP.columns = column

    HR_time = HR_time.fillna(-1)
    SBP_time = SBP_time.fillna(-1)
    count = 0

    i = 0
    column = []
    while i < len(HR_time.columns):
        column.append("HR_offset_%s"%i)
        i += 1
    HR_time.columns = column

    i = 0
    column = []
    while i < len(SBP_time.columns):
        column.append("SBP_offset_%s"%i)
        i += 1
    SBP_time.columns = column


    SBP_size = SBP_res.groupby("patientunitstayid").size()
    HR_size = HR_res.groupby("patientunitstayid").size()

    # length_MSI = #int(int(strsql_timeoffst)/offset)

    MSI = []
    i = 0
    while i < len(SBP.index):
        #one time deal with one patient
        goal = 0
        loop = 0
#         whether index is in DBP SBP HR
        if SBP.index[i] in HR.index.values:
            SIpt = []
            SIpt.append(SBP.index[i])
    #         take DBP columns as basic columns length check loop value under length and success under 9
            while loop < SBP_size.loc[SBP.index[i]] and goal < 1:
                #whether we have every ingredient
                if SBP_time.loc[SBP.index[i]][loop]in HR_time.loc[SBP.index[i]].values:
                    SBP_col_index = loop
                    HR_col_index = int(np.where(HR_time.loc[SBP.index[i]].values == SBP_time.loc[SBP.index[i]][loop])[0][0])
                    #caculate MSI
                    msi = 0
                    pulse = 0
                    MAPp = 0
                    if (float(HR.loc[SBP.index[i]][HR_col_index]) != 0 or -1) and (float(SBP.loc[SBP.index[i]][SBP_col_index]) != 0 or -1): 
                        MAPp = float(SBP.loc[SBP.index[i]][SBP_col_index])
                        if MAPp != 0:
                            msi = float(HR.loc[SBP.index[i]][HR_col_index]) / MAPp
                            SIpt.append(msi)
                            goal += 1
                loop += 1
            MSI.append(SIpt)
        i += 1

    MSI = pd.DataFrame(MSI)
    column = ['patientunitstayid']
    column.append("MSI_%s"%(7 - epoch))
    MSI.columns = column
    MSI.set_index('patientunitstayid',inplace = True)
    MSI = MSI.fillna(-1)
    MSI = pd.DataFrame(MSI,dtype = np.float)
    store.append(MSI)
    epoch -= 1
    print(epoch)

#merge every time period
MSI = store[0].index.values
i = 1
while i < len(store):
    MSI = np.union1d(MSI,store[i].index.values)
    i += 1

MSI = pd.DataFrame(MSI)
MSI.columns = ['patientunitstayid']
MSI.set_index('patientunitstayid',inplace = True)
for i in store:
    MSI = pd.merge(MSI,i,on = 'patientunitstayid',how = 'left')
MSI = MSI.fillna(-1)

i = 0
while i < len(MSI.columns):
    if i == 0:
        j = 1
        while j < len(MSI.columns):
            MSI.loc[((MSI['%s'%MSI.columns[i]] == -1) & (MSI['%s'%MSI.columns[j]] != -1)), '%s'%MSI.columns[i]] = MSI['%s'%MSI.columns[j]]
            j += 1
    elif i < len(MSI.columns) - 1:
        MSI.loc[((MSI['%s'%MSI.columns[i]] == -1) & (MSI['%s'%MSI.columns[i + 1]] != -1)), '%s'%MSI.columns[i]] = (MSI['%s'%MSI.columns[i - 1]] + MSI['%s'%MSI.columns[i + 1]])/2
        MSI.loc[(MSI['%s'%MSI.columns[i]] == -1), '%s'%MSI.columns[i]] = MSI['%s'%MSI.columns[i - 1]]
    else:
        MSI.loc[(MSI['%s'%MSI.columns[i]] == -1), '%s'%MSI.columns[i]] = MSI['%s'%MSI.columns[i - 1]]
    i += 1

print('')
for i in MSI.columns:
    print("%s Missing Rate:"%i,1 - MSI[MSI['%s'%i] != -1]['%s'%i].count()/all_patient)
    print("%s Null Count:"%i,MSI[MSI['%s'%i] == -1]['%s'%i].count())
    print('')

MSI


5
4
3
2
1
0

MSI_1 Missing Rate: 0.3292964180883229
MSI_1 Null Count: 608

MSI_2 Missing Rate: 0.3292964180883229
MSI_2 Null Count: 608

MSI_3 Missing Rate: 0.3292964180883229
MSI_3 Null Count: 608

MSI_4 Missing Rate: 0.3292964180883229
MSI_4 Null Count: 608

MSI_5 Missing Rate: 0.3292964180883229
MSI_5 Null Count: 608

MSI_6 Missing Rate: 0.3292964180883229
MSI_6 Null Count: 608



,MSI_1,MSI_2,MSI_3,MSI_4,MSI_5,MSI_6
patientunitstayid,,,,,,
141168,1.707317,1.707317,1.707317,1.707317,1.707317,1.707317
141203,0.703390,0.653846,0.625943,0.598039,0.880415,1.162791
141227,0.850000,0.812387,0.774775,0.792793,0.810811,0.784000
141229,0.540881,0.540881,0.540881,0.540881,0.540881,0.540881
141284,0.800000,0.800000,0.800000,0.800000,0.800000,0.800000
...,...,...,...,...,...,...
3353216,0.648649,0.648649,0.648649,0.648649,0.648649,0.575221
3353226,0.570248,0.570248,0.570248,0.570248,0.570248,0.570248
3353235,0.633803,0.633803,0.633803,0.633803,0.633803,0.633803


In [371]:
#show how to concat two dataframe,use for loop to connect all dataframe(must start with age_res,since its missing rate is 0)
NOAKIPT = pd.DataFrame()
NOAKIPT = pd.merge(age,nettotal,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,dialysis,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,potassium,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,sodium,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,Hct,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,BUN,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,pH,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,OI,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,temperature,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,urine,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,MAP,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,creatinine,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,glucose,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,albumin,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,MSI,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,weight,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,condition,on='patientunitstayid',how='left')
NOAKIPT['aki'] = 0
NOAKIPT = NOAKIPT.fillna(-1)
NOAKIPT = NOAKIPT.loc[NOAKIPT['IsDialysis'] != 1]
NOAKIPT.drop('IsDialysis',axis = 1,inplace = True)
NOAKIPT.to_csv('NOAKIPT_RNN_6.csv', encoding='utf-8')

NOAKIPT


,age,nettotal,potassium,potassium_1,potassium_2,potassium_3,potassium_4,potassium_5,potassium_6,sodium,...,MSI_4,MSI_5,MSI_6,weight,cirrhosis,gender,hepaticFailure,ventday1,diabetes,aki
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141168,70.0,0.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,1.707317,1.707317,1.707317,84.3,0.0,1.0,0.0,0.0,0.0,0
141178,52.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,54.4,0.0,1.0,0.0,0.0,0.0,0
141179,52.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
141194,68.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,73.9,0.0,0.0,0.0,0.0,1.0,0
141196,71.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353216,50.0,-2138.56,5.1,5.1000,5.1000,5.1000,5.1000,5.1000,5.1000,143.0,...,0.648649,0.648649,0.575221,55.4,0.0,1.0,0.0,1.0,0.0,0
3353226,79.0,1.41,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,4.8000,136.0,...,0.570248,0.570248,0.570248,58.4,0.0,1.0,0.0,1.0,1.0,0
3353235,50.0,0.00,4.1,4.1000,4.1000,4.1000,4.1000,4.1000,4.1000,137.0,...,0.633803,0.633803,0.633803,90.0,0.0,0.0,0.0,0.0,0.0,0


In [372]:
AKIPT

,age,nettotal,potassium,potassium_1,potassium_2,potassium_3,potassium_4,potassium_5,potassium_6,sodium,...,MSI_4,MSI_5,MSI_6,weight,cirrhosis,gender,hepaticFailure,ventday1,diabetes,aki
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141266,73.0,63.22,5.2,5.2000,5.2000,5.2000,5.2000,5.2000,5.2000,135.0,...,0.810345,0.910891,0.960000,120.40,0.0,0.0,0.0,0.0,1.0,1
141304,70.0,1253.00,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,5.8000,141.0,...,0.854015,0.854015,0.991597,-1.00,0.0,0.0,0.0,1.0,0.0,1
141314,45.0,150.00,8.0,8.0000,8.0000,8.0000,8.0000,8.0000,8.0000,158.0,...,1.545455,1.545455,1.545455,-1.00,0.0,0.0,0.0,1.0,0.0,1
141448,65.0,0.00,2.8,2.8000,2.8000,2.8000,2.8000,2.8000,2.8000,138.0,...,0.860927,0.860927,0.851351,79.80,0.0,0.0,0.0,0.0,0.0,1
141470,58.0,0.00,4.3,4.3000,4.3000,4.3000,4.3000,4.3000,4.3000,131.0,...,0.947761,0.947761,1.120000,-1.00,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353194,51.0,-302.00,3.4,3.4000,3.3500,3.3000,3.5000,3.5500,3.6000,139.0,...,1.125000,0.990385,1.147727,63.05,0.0,1.0,0.0,1.0,1.0,1
3353197,66.0,2404.60,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,4.8000,132.0,...,0.706897,0.627273,0.673077,71.50,0.0,1.0,0.0,0.0,0.0,1
3353199,66.0,17.44,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,71.50,-1.0,-1.0,-1.0,-1.0,-1.0,1


In [373]:
NOAKIPT

,age,nettotal,potassium,potassium_1,potassium_2,potassium_3,potassium_4,potassium_5,potassium_6,sodium,...,MSI_4,MSI_5,MSI_6,weight,cirrhosis,gender,hepaticFailure,ventday1,diabetes,aki
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141168,70.0,0.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,1.707317,1.707317,1.707317,84.3,0.0,1.0,0.0,0.0,0.0,0
141178,52.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,54.4,0.0,1.0,0.0,0.0,0.0,0
141179,52.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
141194,68.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,73.9,0.0,0.0,0.0,0.0,1.0,0
141196,71.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353216,50.0,-2138.56,5.1,5.1000,5.1000,5.1000,5.1000,5.1000,5.1000,143.0,...,0.648649,0.648649,0.575221,55.4,0.0,1.0,0.0,1.0,0.0,0
3353226,79.0,1.41,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,4.8000,136.0,...,0.570248,0.570248,0.570248,58.4,0.0,1.0,0.0,1.0,1.0,0
3353235,50.0,0.00,4.1,4.1000,4.1000,4.1000,4.1000,4.1000,4.1000,137.0,...,0.633803,0.633803,0.633803,90.0,0.0,0.0,0.0,0.0,0.0,0


In [199]:


for i in AKIPT.columns:
    print("%s Missing Rate:"%i,1 - AKIPT[AKIPT['%s'%i] != -1]['%s'%i].count()/aki_patient)
AKIPT


age Missing Rate: 0.09309202141218453
nettotal Missing Rate: 0.39112923782819276
potassium Missing Rate: 0.22880448636247774
potassium_1 Missing Rate: 0.22880448636247774
potassium_2 Missing Rate: 0.22880448636247774
potassium_3 Missing Rate: 0.22880448636247774
potassium_4 Missing Rate: 0.22880448636247774
sodium Missing Rate: 0.2275809329594698
sodium_1 Missing Rate: 0.2275809329594698
sodium_2 Missing Rate: 0.2275809329594698
sodium_3 Missing Rate: 0.2275809329594698
sodium_4 Missing Rate: 0.2275809329594698
Hct Missing Rate: 0.2528167218965077
Hct_1 Missing Rate: 0.2525618149375478
Hct_2 Missing Rate: 0.2525618149375478
Hct_3 Missing Rate: 0.2525618149375478
Hct_4 Missing Rate: 0.2525618149375478
BUN Missing Rate: 0.2423655365791486
BUN_1 Missing Rate: 0.24144787152689273
BUN_2 Missing Rate: 0.24144787152689273
BUN_3 Missing Rate: 0.24144787152689273
BUN_4 Missing Rate: 0.24144787152689273
pH Missing Rate: 0.6203925567167984
pH_1 Missing Rate: 0.6194239102727505
pH_2 Missing Rate: 

,age,nettotal,potassium,potassium_1,potassium_2,potassium_3,potassium_4,sodium,sodium_1,sodium_2,...,MSI_2,MSI_3,MSI_4,weight,cirrhosis,gender,hepaticFailure,ventday1,diabetes,aki
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141266,73.0,63.22,5.2,5.2000,5.2000,5.2000,5.2000,135.0,135.0000,135.0000,...,0.777778,0.839695,0.833333,120.40,0.0,0.0,0.0,0.0,1.0,1
141304,70.0,1253.00,4.8,4.8000,4.8000,4.8000,5.8000,141.0,141.0000,141.0000,...,0.806897,1.027273,1.045455,-1.00,0.0,0.0,0.0,1.0,0.0,1
141314,45.0,150.00,8.0,8.0000,8.0000,8.0000,8.0000,158.0,158.0000,158.0000,...,1.612903,1.132530,0.739130,-1.00,0.0,0.0,0.0,1.0,0.0,1
141448,65.0,0.00,2.8,2.8000,2.8000,2.8000,2.8000,138.0,138.0000,138.0000,...,0.867052,0.729412,0.731707,79.80,0.0,0.0,0.0,0.0,0.0,1
141470,58.0,0.00,4.3,4.3000,4.3000,4.3000,4.3000,131.0,131.0000,131.0000,...,1.115385,1.120000,1.098039,-1.00,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353194,51.0,-302.00,3.4,3.4000,3.3000,3.5000,3.6000,139.0,139.0000,141.0000,...,0.828283,0.833333,0.794393,63.05,0.0,1.0,0.0,1.0,1.0,1
3353197,66.0,2404.60,4.8,4.8000,4.8000,4.8000,4.8000,132.0,132.0000,132.0000,...,0.759615,0.669291,0.725664,71.50,0.0,1.0,0.0,0.0,0.0,1
3353199,66.0,17.44,-1.0,-1,-1,-1,-1,-1.0,-1,-1,...,-1.000000,-1.000000,-1.000000,71.50,-1.0,-1.0,-1.0,-1.0,-1.0,1


In [374]:
HC_NOAKIPT = NOAKIPT.loc[(NOAKIPT['potassium'] != -1) & (NOAKIPT['sodium'] != -1) & (NOAKIPT['Hct'] != -1) & (NOAKIPT['BUN'] != -1) & (NOAKIPT['temperature'] != -1) & (NOAKIPT['MAP_1'] != -1) & (NOAKIPT['creatinine'] != -1) & (NOAKIPT['glucose'] != -1) & (NOAKIPT['albumin'] != -1) & (NOAKIPT['MSI_1'] != -1)]
NC_NOAKIPT = NOAKIPT.loc[NOAKIPT['creatinine'] == -1]
partof_HC_NOAKIPT = HC_NOAKIPT.sample(n=13165,random_state=123,axis=0)
partof_NC_NOAKIPT = NC_NOAKIPT.sample(n=4625,random_state=123,axis=0)

Noaki = partof_HC_NOAKIPT.append(partof_NC_NOAKIPT)

TrainPT = AKIPT.append(Noaki)

TrainPT.to_csv('TrainPT_RNN_6.csv', encoding='utf-8')


AllPT = pd.DataFrame()
AllPT = AKIPT.append(NOAKIPT)
AllPT.to_csv('AllPT_RNN_6.csv', encoding='utf-8')

TrainPT


,age,nettotal,potassium,potassium_1,potassium_2,potassium_3,potassium_4,potassium_5,potassium_6,sodium,...,MSI_4,MSI_5,MSI_6,weight,cirrhosis,gender,hepaticFailure,ventday1,diabetes,aki
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141266,73.0,63.22,5.2,5.2000,5.2000,5.2000,5.2000,5.2000,5.2000,135.0,...,0.810345,0.910891,0.960000,120.4,0.0,0.0,0.0,0.0,1.0,1
141304,70.0,1253.00,4.8,4.8000,4.8000,4.8000,4.8000,4.8000,5.8000,141.0,...,0.854015,0.854015,0.991597,-1.0,0.0,0.0,0.0,1.0,0.0,1
141314,45.0,150.00,8.0,8.0000,8.0000,8.0000,8.0000,8.0000,8.0000,158.0,...,1.545455,1.545455,1.545455,-1.0,0.0,0.0,0.0,1.0,0.0,1
141448,65.0,0.00,2.8,2.8000,2.8000,2.8000,2.8000,2.8000,2.8000,138.0,...,0.860927,0.860927,0.851351,79.8,0.0,0.0,0.0,0.0,0.0,1
141470,58.0,0.00,4.3,4.3000,4.3000,4.3000,4.3000,4.3000,4.3000,131.0,...,0.947761,0.947761,1.120000,-1.0,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3026540,67.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,50.8,0.0,1.0,0.0,0.0,0.0,0
2422363,71.0,-300.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,0.484848,0.484848,0.395973,93.0,0.0,0.0,0.0,0.0,0.0,0
731219,70.0,-1.00,-1.0,-1,-1,-1,-1,-1,-1,-1.0,...,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0


In [379]:
data = pd.read_csv('TrainPT_RNN_6.csv', encoding='utf-8')
data = data.fillna(-1)
dataS = data.copy()
dataO = data.copy()

dataS.drop('age',axis = 1,inplace = True)
dataS.drop('nettotal',axis = 1,inplace = True)
dataS.drop('weight',axis = 1,inplace = True)
dataS.drop('cirrhosis',axis = 1,inplace = True)
dataS.drop('gender',axis = 1,inplace = True)
dataS.drop('hepaticFailure',axis = 1,inplace = True)
dataS.drop('ventday1',axis = 1,inplace = True)
dataS.drop('diabetes',axis = 1,inplace = True)
dataS.drop('aki',axis = 1,inplace = True)
dataS.drop('OI_0',axis = 1,inplace = True)

for i in dataS.columns:
    if i != 'patientunitstayid':
        dataO.drop('%s'%i,axis = 1,inplace = True)

Newtrain = []

i = 0
while i < len(data['patientunitstayid']):
    j = 0
    while j < int(int(strsql_timeoffst)/offset):
        Newtrain.append(data['patientunitstayid'].iloc[i])
        j += 1
    i += 1

Newtrain = pd.DataFrame(Newtrain)
Newtrain.columns = ['patientunitstayid']
Newtrain = Newtrain.merge(dataO,on = 'patientunitstayid',how = 'inner')

#after adding value
dataS.drop('potassium',axis = 1,inplace = True)
dataS.drop('sodium',axis = 1,inplace = True)
dataS.drop('Hct',axis = 1,inplace = True)
dataS.drop('BUN',axis = 1,inplace = True)
dataS.drop('pH',axis = 1,inplace = True)
dataS.drop('temperature',axis = 1,inplace = True)
dataS.drop('creatinine',axis = 1,inplace = True)
dataS.drop('glucose',axis = 1,inplace = True)
dataS.drop('albumin',axis = 1,inplace = True)

#transform
Newtrain['potassium'] = 0
Newtrain['sodium'] = 0
Newtrain['Hct'] = 0
Newtrain['BUN'] = 0
Newtrain['pH'] = 0
Newtrain['temperature'] = 0
Newtrain['urine'] = 0
Newtrain['MAP'] = 0
Newtrain['creatinine'] = 0
Newtrain['glucose'] = 0
Newtrain['albumin'] = 0
Newtrain['MSI'] = 0
dataS.set_index('patientunitstayid',inplace = True)

i = 0
potassium = []
sodium = []
Hct = []
BUN = []
pH = []
temperature = []
urine = []
MAP = []
creatinine = []
glucose = []
albumin = []
MSI = []
while i < len(Newtrain) - int(int(strsql_timeoffst)/offset) -1:
    j = 0
    while j < int(int(strsql_timeoffst)/offset):
        potassium.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)
    while j < int(int(strsql_timeoffst)/offset)*2:
        sodium.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*2
    while j < int(int(strsql_timeoffst)/offset)*3:
        Hct.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*3
    while j < int(int(strsql_timeoffst)/offset)*4:
        BUN.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*4
    while j < int(int(strsql_timeoffst)/offset)*5:
        pH.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*5
    while j < int(int(strsql_timeoffst)/offset)*6:
        temperature.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*6
    while j < int(int(strsql_timeoffst)/offset)*7:
        urine.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*7
    while j < int(int(strsql_timeoffst)/offset)*8:
        MAP.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*8
    while j < int(int(strsql_timeoffst)/offset)*9:
        creatinine.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*9
    while j < int(int(strsql_timeoffst)/offset)*10:
        glucose.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*10
    while j < int(int(strsql_timeoffst)/offset)*11:
        albumin.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    j = int(int(strsql_timeoffst)/offset)*11
    while j < int(int(strsql_timeoffst)/offset)*12:
        MSI.append(dataS.loc[Newtrain.iloc[i]['patientunitstayid']][dataS.columns[j]])
        j += 1
    i += int(int(strsql_timeoffst)/offset)

Newtrain['potassium'] = pd.DataFrame(potassium)
Newtrain['sodium'] = pd.DataFrame(sodium)
Newtrain['Hct'] = pd.DataFrame(Hct)
Newtrain['BUN'] = pd.DataFrame(BUN)
Newtrain['pH'] = pd.DataFrame(pH)
Newtrain['temperature'] = pd.DataFrame(temperature)
Newtrain['urine'] = pd.DataFrame(urine)
Newtrain['MAP'] = pd.DataFrame(MAP)
Newtrain['creatinine'] = pd.DataFrame(creatinine)
Newtrain['glucose'] = pd.DataFrame(glucose)
Newtrain['albumin'] = pd.DataFrame(albumin)
Newtrain['MSI'] = pd.DataFrame(MSI)

Newtrain.loc[Newtrain['cirrhosis'] == -1, 'cirrhosis'] = 0
Newtrain.loc[Newtrain['hepaticFailure'] == -1, 'hepaticFailure'] = 0
Newtrain.loc[Newtrain['ventday1'] == -1, 'ventday1'] = 0
Newtrain.loc[Newtrain['diabetes'] == -1, 'diabetes'] = 0
Newtrain.loc[Newtrain['aki'] == -1, 'aki'] = 0
Newtrain = Newtrain.fillna(-1)

Newtrain


,patientunitstayid,age,nettotal,OI_0,weight,cirrhosis,gender,hepaticFailure,ventday1,diabetes,...,Hct,BUN,pH,temperature,urine,MAP,creatinine,glucose,albumin,MSI
0,141266,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,...,33.4,29.0,7.18,36.3,-1.0,75.666667,1.9,423.0,3.6,0.916667
1,141266,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,...,33.4,29.0,7.18,36.3,-1.0,75.666667,1.9,423.0,3.6,0.916667
2,141266,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,...,33.4,29.0,7.18,36.3,-1.0,75.666667,1.9,423.0,3.6,0.916667
3,141266,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,...,33.4,29.0,7.36,36.2,-1.0,75.666667,1.9,423.0,3.6,0.810345
4,141266,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,...,33.4,29.0,7.36,36.2,-1.0,75.666667,1.9,423.0,3.6,0.910891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213475,1547119,54.0,40.00,-1.0,119.7,0.0,-1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213476,1547119,54.0,40.00,-1.0,119.7,0.0,-1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213477,1547119,54.0,40.00,-1.0,119.7,0.0,-1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213478,1547119,54.0,40.00,-1.0,119.7,0.0,-1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [381]:
Newtrain.drop('patientunitstayid',axis = 1,inplace = True)
Newtrain.to_csv('Real_RNN_S_6.csv', encoding='utf-8')

Newtrain


,age,nettotal,OI_0,weight,cirrhosis,gender,hepaticFailure,ventday1,diabetes,aki,...,Hct,BUN,pH,temperature,urine,MAP,creatinine,glucose,albumin,MSI
0,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,1,...,33.4,29.0,7.18,36.3,-1.0,75.666667,1.9,423.0,3.6,0.916667
1,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,1,...,33.4,29.0,7.18,36.3,-1.0,75.666667,1.9,423.0,3.6,0.916667
2,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,1,...,33.4,29.0,7.18,36.3,-1.0,75.666667,1.9,423.0,3.6,0.916667
3,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,1,...,33.4,29.0,7.36,36.2,-1.0,75.666667,1.9,423.0,3.6,0.810345
4,73.0,63.22,291.0,120.4,0.0,0.0,0.0,0.0,1.0,1,...,33.4,29.0,7.36,36.2,-1.0,75.666667,1.9,423.0,3.6,0.910891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213475,54.0,40.00,-1.0,119.7,0.0,-1.0,0.0,0.0,0.0,0,...,-1.0,-1.0,-1.00,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.000000
213476,54.0,40.00,-1.0,119.7,0.0,-1.0,0.0,0.0,0.0,0,...,-1.0,-1.0,-1.00,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.000000
213477,54.0,40.00,-1.0,119.7,0.0,-1.0,0.0,0.0,0.0,0,...,-1.0,-1.0,-1.00,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.000000
213478,54.0,40.00,-1.0,119.7,0.0,-1.0,0.0,0.0,0.0,0,...,-1.0,-1.0,-1.00,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.000000


In [ ]:
print(dataS.columns[0])
i = 0
# dataS.set_index('patientunitstayid',inplace = True)
dataS.loc[Newtrain.iloc[i]['patientunitstayid']]['%s'%(dataS.columns[13])]
 = dataS.loc[Newtrain.iloc[i]['patientunitstayid']]['%s_%s'%]

In [210]:
import pandas as pd
import numpy as np
import math
import statistics
data = pd.read_csv('Real_RNN_S.csv', encoding='utf-8')

data.drop('Unnamed: 0',axis = 1,inplace = True)
#no record store as don't have
data.loc[data['cirrhosis'] == -1, 'cirrhosis'] = 0
data.loc[data['hepaticFailure'] == -1, 'hepaticFailure'] = 0
data.loc[data['ventday1'] == -1, 'ventday1'] = 0
data.loc[data['diabetes'] == -1, 'diabetes'] = 0
data.loc[data['aki'] == -1, 'aki'] = 0
data.replace(-1, 'nan', inplace = True)
data = data[data!='nan']
mean = []
for i in data.columns:
    if i != 'aki' and i != 'gender':
        mean.append(data[i].mean())
#         data.loc[data[i] == 'nan', i] = data[i].mean()
        print(i,'\t',data[i].mean())
data = data.fillna(-1)
j = 0
for i in data.columns:
    if i != 'aki' and i != 'gender':
        data.loc[data[i] == -1, i] = mean[j]
        j += 1
data.to_csv('Real_RNN_NS.csv', encoding='utf-8')    

for i in data.columns:
    print("%s Missing Rate:"%i,1 - data[data['%s'%i] != -1]['%s'%i].count()/len(data))
    
data


age 	 64.27156190262004
nettotal 	 -63.741491190920094
OI_0 	 215.04163749968438
weight 	 85.25128757782457
cirrhosis 	 0.023917931422147275
hepaticFailure 	 0.021781899943788645
ventday1 	 0.25401911186059584
diabetes 	 0.22987633501967397
potassium 	 4.356309648754524
sodium 	 137.49484475149728
Hct 	 35.16326498200137
BUN 	 36.93459452479343
pH 	 7.331370093476909
temperature 	 36.89361949929608
urine 	 447.343101939999
MAP 	 83.9594079141644
creatinine 	 2.08981678402067
glucose 	 161.73003663334177
albumin 	 3.1943799200169014
MSI 	 0.7960632231401856
age Missing Rate: 0.0
nettotal Missing Rate: 0.0
OI_0 Missing Rate: 0.0
weight Missing Rate: 0.0
cirrhosis Missing Rate: 0.0
gender Missing Rate: 0.08116919617762786
hepaticFailure Missing Rate: 0.0
ventday1 Missing Rate: 0.0
diabetes Missing Rate: 0.0
aki Missing Rate: 0.0
potassium Missing Rate: 0.0
sodium Missing Rate: 0.0
Hct Missing Rate: 0.0
BUN Missing Rate: 0.0
pH Missing Rate: 0.0
temperature Missing Rate: 0.0
urine Missing 

,age,nettotal,OI_0,weight,cirrhosis,gender,hepaticFailure,ventday1,diabetes,aki,...,Hct,BUN,pH,temperature,urine,MAP,creatinine,glucose,albumin,MSI
0,73.0,63.220000,291.000000,120.400000,0.0,0.0,0.0,0.0,1.0,1,...,33.400000,29.000000,7.18000,36.300000,447.343102,75.666667,1.900000,423.000000,3.60000,0.916667
1,73.0,63.220000,291.000000,120.400000,0.0,0.0,0.0,0.0,1.0,1,...,33.400000,29.000000,7.18000,36.300000,447.343102,75.666667,1.900000,423.000000,3.60000,0.777778
2,73.0,63.220000,291.000000,120.400000,0.0,0.0,0.0,0.0,1.0,1,...,33.400000,29.000000,7.36000,36.200000,447.343102,75.666667,1.900000,423.000000,3.60000,0.839695
3,73.0,63.220000,291.000000,120.400000,0.0,0.0,0.0,0.0,1.0,1,...,33.400000,29.000000,7.36000,36.200000,447.343102,75.666667,1.900000,423.000000,3.60000,0.833333
4,70.0,1253.000000,215.041637,85.251288,0.0,0.0,0.0,1.0,0.0,1,...,28.800000,41.000000,7.32000,36.800000,447.343102,72.000000,2.340000,173.000000,3.00000,0.854015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142315,59.0,-63.741491,464.285714,104.300000,0.0,0.0,0.0,0.0,1.0,0,...,35.163265,36.934595,7.42000,36.893619,447.343102,86.000000,2.089817,161.730037,3.19438,0.708333
142316,54.0,40.000000,215.041637,119.700000,0.0,-1.0,0.0,0.0,0.0,0,...,35.163265,36.934595,7.33137,36.700000,200.000000,127.666667,2.089817,161.730037,3.19438,0.705882
142317,54.0,40.000000,215.041637,119.700000,0.0,-1.0,0.0,0.0,0.0,0,...,35.163265,36.934595,7.33137,36.700000,447.343102,127.666667,2.089817,161.730037,3.19438,0.705882
142318,54.0,40.000000,215.041637,119.700000,0.0,-1.0,0.0,0.0,0.0,0,...,35.163265,36.934595,7.33137,36.700000,447.343102,127.666667,2.089817,161.730037,3.19438,0.705882


In [ ]:
dataS
# i = 1
# while i < len(dataS.columns)-3:
#     print(dataS.columns[i],dataS.columns[i+1],dataS.columns[i+2],dataS.columns[i+3])
#     i += 4
